### Software setup


In [ ]:
# Mount to Google Drive

import os, sys
from google.colab import drive
drive.mount('/content/mnt', force_remount=True)
nb_path = '/content/notebooks'
os.symlink('/content/mnt/My Drive/Colab Notebooks', nb_path)
sys.path.insert(0, nb_path)  # or append(nb_path)

Mounted at /content/mnt


In [ ]:
# Install WRDS package
!pip install --target=$nb_path wrds

     |████████████████████████████████| 3.4 MB 8.5 MB/s 
     |████████████████████████████████| 15.7 MB 61 kB/s 
     |████████████████████████████████| 1.6 MB 32.6 MB/s 
     |████████████████████████████████| 11.3 MB 72.9 MB/s 
     |████████████████████████████████| 247 kB 65.2 MB/s 
     |████████████████████████████████| 503 kB 61.9 MB/s 
     |████████████████████████████████| 150 kB 64.1 MB/s 
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.6.0 requires numpy~=1.19.2, but you have numpy 1.21.4 which is incompatible.
tensorflow 2.6.0 requires six~=1.15.0, but you have six 1.16.0 which is incompatible.
tensorflow 2.6.0 requires typing-extensions~=3.7.4, but you have typing-extensions 3.10.0.2 which is incompatible.
google-colab 1.0.0 requires pandas~=1.1.0; python_version >= "3.0", but you have pandas 1.3.4 which is incompatible.
google-colab

In [ ]:
# Installing WRDS api
!pip install wrds

  Using cached wrds-3.1.1-py3-none-any.whl (12 kB)
  Using cached mock-4.0.3-py3-none-any.whl (28 kB)
  Using cached psycopg2_binary-2.9.1-cp37-cp37m-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (3.4 MB)


### Bloomberg API installation code 

In [ ]:
# Installing blp (Pythonic interface for Bloomberg Open API)
# https://github.com/matthewgilbert/blp

!pip install blp

In [ ]:
# Installing Bloomberg API 
# See https://www.bloomberg.com/professional/support/api-library/
!pip install blpapi --index-url=https://bloomberg.bintray.com/pip/simple
!pip install xbbg

Looking in indexes: https://bloomberg.bintray.com/pip/simple
ERROR: Could not find a version that satisfies the requirement blpapi (from versions: none)
ERROR: No matching distribution found for blpapi
     |████████████████████████████████| 52 kB 1.2 MB/s 
  Using cached pytz-2021.3-py2.py3-none-any.whl (503 kB)
     |████████████████████████████████| 109 kB 17.5 MB/s 
     |████████████████████████████████| 546 kB 57.5 MB/s 
  Attempting uninstall: pytz
    Found existing installation: pytz 2018.9
    Uninstalling pytz-2018.9:
      Successfully uninstalled pytz-2018.9


## WRDS API Installation

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import wrds
import datetime
import json

db = wrds.Connection()

db.create_pgpass_file()

Enter your WRDS username [root]:ani09
Enter your password:··········
WRDS recommends setting up a .pgpass file.
Create .pgpass file now [y/n]?: y
Created .pgpass file successfully.
Loading library list...
Done


### WRDS setup

In [ ]:
# Creating a function to make querying a bit easier to type.
def query_wrds(query, user):
    with wrds.Connection(wrds_username=user) as db:
        data = db.raw_sql(query)
    return data

In [ ]:
# names_in_wrds_df = db.raw_sql("SELECT * FROM tr_esg.wrds_company_info")
# jeopardy_qa_df = pd.read_csv('/content/mnt/MyDrive/COMP_upd.csv')
# companies_all_df = jeopardy_qa_df.drop(columns=['Unnamed: 0'])

In [ ]:
# russel_1000_df=pd.read_csv("/content/russel_1000.csv").drop(columns=['Unnamed: 0'])
russel_leftover_df=pd.read_csv("/content/mnt/MyDrive/company_has_org.csv").drop(columns=['Unnamed: 0'])

In [ ]:
# basic_100_df=pd.read_csv("/content/try.csv").drop(columns=['Unnamed: 0'])

In [ ]:
russel_leftover_df

In [ ]:
import sqlite3

conn = sqlite3.connect('local.db')

russel_1000_df.to_sql("companies", conn, if_exists="replace", index=False)

pd.read_sql_query('select * from companies', conn)

In [ ]:
dd = pd.DataFrame()

for row in basic_100_df.iterrows():
  symbol = (row[1]['symbol'])
  cusip = (row[1]['cusip'])
  tot = db.raw_sql(f"SELECT * FROM tr_esg.wrds_company_info WHERE cusip='{cusip[:-1]}'")
  if (len(tot)) == 0:
    tot = db.raw_sql(f"SELECT * FROM tr_esg.wrds_company_info WHERE tick='{symbol}'")
  if (len(tot)) == 0:
    print("STILL", symbol)
  dd = dd.append(tot)


In [ ]:
company_has_org_df = dd[['orgid', 'orgname', 'tick', 'cusip']]

In [ ]:
company_has_org_df = company_has_org_df.dropna()

In [ ]:
company_has_org_df.to_csv("company_has_org.csv")

In [ ]:
db.raw_sql(f"SELECT * FROM tr_esg.wrds_esg_catscores LIMIT 5")

In [ ]:
# List all libraries in WRDS
# db.list_libraries().sort()
# db.list_libraries()

In [ ]:
# List all tables within a library (tr_esg is Refinitiv's ESG data library)
total = db.list_tables(library='tr_esg')
total

['a4catdata',
 'a4cmpchg',
 'a4cmpfdata',
 'a4cmpidata',
 'a4cmpinfo',
 'a4cmpsdata',
 'a4cmpsummdata',
 'a4code',
 'a4desc',
 'a4driverdata',
 'a4indaggrdata',
 'a4item',
 'a4memfdata',
 'a4memidata',
 'a4memsdata',
 'a4outcomedata',
 'wrds_company_cg',
 'wrds_company_ec',
 'wrds_company_en',
 'wrds_company_info',
 'wrds_company_smry',
 'wrds_company_so',
 'wrds_esg_boardmem',
 'wrds_esg_catscores',
 'wrds_esg_drivers',
 'wrds_esg_indagg',
 'wrds_esg_outcomes']

In [ ]:
russel_leftover_df

In [ ]:
db.raw_sql(f"SELECT * FROM tr_esg.wrds_esg_catscores LIMIT 4")

,orgid,fisyear,cdflag,feeddate,fyenddate,cg_bd_bf,cg_bd_bs,cg_bd_cp,cg_in_vs,cg_sh_sr,ec_ma_pe,ec_pr_sl,ec_re_cl,en_en_er,en_en_pi,en_en_rr,so_cu_pr,so_so_co,so_so_hr,so_wo_do,so_wo_eq,so_wo_hs,so_wo_td,score,cscore,controversiesscore,resourceusescore,emissionsscore,innovationscore,workforcescore,humanrightsscore,communityscore,productrespscore,managementscore,shareholdersscore,csrstrategyscore
0,10014.0,2017.0,500.0,2021-09-12,2017-12-31,0.133968,0.252110,0.710553,0.450873,0.451312,0.371852,0.565041,0.140463,0.306771,0.594566,0.528954,0.204667,0.134683,0.578074,0.556046,0.305888,0.764517,0.703794,0.592532,0.592532,1.0,0.218750,0.415441,0.699074,0.724390,0.089286,0.568293,0.309353,0.916667,0.972222,0.083333
1,10014.0,2018.0,500.0,2021-09-12,2018-12-31,0.225599,0.394001,0.395091,0.437932,0.527877,0.643755,0.202724,0.149223,0.595474,0.618164,0.570780,0.184485,0.122413,0.565440,0.707875,0.633625,0.831174,0.829689,0.568982,0.568982,1.0,0.216292,0.546784,0.710145,0.667373,0.054167,0.593220,0.330508,0.783333,0.916667,0.083333
2,10014.0,2019.0,500.0,2021-09-12,2019-12-31,0.308772,0.432291,0.552788,0.399387,0.531437,0.842687,0.828948,0.161277,0.387259,0.800864,0.439381,0.288695,0.155909,0.529828,0.864055,0.926474,0.819146,0.788462,0.546237,0.546237,1.0,0.021645,0.265625,0.703390,0.776451,0.027273,0.573379,0.314050,0.863636,0.954545,0.106061
3,10014.0,2020.0,510.0,2021-09-12,None,0.286328,0.621999,0.563088,0.363382,0.514689,0.904467,0.257841,0.099481,0.351222,0.777933,0.405672,0.260570,0.142922,0.504687,0.825979,0.923178,0.767007,0.762257,0.535270,0.535270,1.0,0.014228,0.240741,0.685930,0.754125,0.027778,0.498350,0.285985,0.897059,0.955882,0.102941


In [ ]:
# db.raw_sql(f"SELECT * FROM tr_esg.a4cmpinfo LIMIT 5").head(10)

In [ ]:
dd = pd.DataFrame()

for idx, name_ticker in russel_leftover_df.iterrows():
  orgid = (name_ticker.orgid)
  orgname = (name_ticker.orgname)
  tick = (name_ticker.tick)
  cusip = (name_ticker.cusip)

  tot = db.raw_sql(f"SELECT * FROM tr_esg.wrds_esg_catscores WHERE orgid='{orgid}' AND (fisyear='2020' OR fisyear='2021')")
  dd = dd.append(tot)

dd

In [ ]:
dd=dd.reset_index()
dd=dd.drop(columns=['level_0', 'index'])
dd

In [ ]:
dd.to_csv('orgid_to_allESGcatscores.csv')

In [ ]:
db.raw_sql(f"SELECT *, {count} as count FROM tr_esg.wrds_company_info WHERE REPLACE(REPLACE(UPPER(orgname),'`',''),',','') LIKE '%FIDELITY%'").head(10)

In [ ]:
import re
import copy
dd = pd.DataFrame()

for idx, name_ticker in companies_all_df.iterrows():
  df_to_ad = pd.DataFrame()
  name=name_ticker[0]
  og_name=name[:]
  if (name == 'Pacific Gas and Electric Company'): name = 'PG&E Corporation'
  if (name == 'Fannie Mae'): name = 'Federal National Mortgage Association'
  if (name == 'The Blackstone Group'): name = 'Blackstone Inc'
  if (name == 'Erie Insurance Group'): name = 'Erie Indemnity'
  if (name == 'Post Holdings'): name = 'Post Holdings Inc'
  if (name == 'L Brands'): name = 'Bath & Body Works Inc'
  if (name == 'U.S. Bancorp'): name = 'US Bancorp'
  if (name == 'LabCorp'): name = 'Laboratory Corp. Of America Holdings'
  if (name.split(" ")[0] == 'U.S.'): 
    name = "United States " + " ".join(name.split(" ")[1:])
  if (name == 'John Deere'): name = 'Deere'
  if (name == 'C. H. Robinson'): name = 'CH Robinson'
  if ("Corp" in name.split(" ")[-1] and name not in ['McKesson Corporation', 'Olin Corporation', 'APA Corporation', 'Lear Corporation', 'Fox Corporation', 'AES Corporation', 'News Corp', 'PPL Corporation']):
    name = ' '.join(name.split(" ")[:-1])
  if ("Corporation" in name.split(" ")[-1]):
    name = ' '.join(name.split(" ")[:-1])+" Corp"
  updated_name = name.replace("'", "")
  if ("Reilly" in updated_name): updated_name = "Reilly"
  updated_name = updated_name.replace("`", "")
  updated_name = updated_name.replace(",", "")
  if ("com" not in updated_name.split('.')):
    updated_name = updated_name.replace(".", "")
  if (updated_name[-1]=='.'): updated_name=updated_name[:-1]
  if (updated_name[:3]=='The'): updated_name=updated_name[3:]
  if (updated_name=='CenturyLink'): updated_name="Lumen"
  if (len(updated_name)==3):
    df_to_ad = db.raw_sql(f"SELECT * FROM tr_esg.wrds_company_info WHERE tick LIKE '%{updated_name.upper()}%'").head(1)
  if (updated_name == 'Meta Platforms Inc'): continue
  if (name in ['Dow Chemical Company', 'CHS Inc.', 'Freddie Mac','Kohlberg Kravis Roberts', 'State Farm', 'KeyBank', 'DCP Midstream Partners']): continue

  broken = updated_name.split(" ")
  new_name=[]
  for part in broken:
    if len(part)==2 and part[-1]=='.': continue
    new_name.append(part)
    
  updated_name = ' '.join(new_name)

  updated_lowercase=' '.join(re.sub( r"([A-Z])", r" \1", updated_name).split())
  updated_second=' '.join(re.sub( r"([A-Z])", r" \1", updated_name).split())
  updated_third=' '.join(re.sub( r"([A-Z])", r" \1", updated_name).split()[1:2])
  updated_2quarter=' '.join(re.sub( r"([A-Z])", r" \1", updated_name).split()[0:2])
  updated_2half=' '.join(re.sub( r"([A-Z])", r" \1", updated_name).split()[0:1])
  og_name=og_name.replace("'","")
  ticker=name_ticker[1]
  if (df_to_ad.shape[0] == 0):
    df_to_ad = db.raw_sql(f"SELECT *,'{og_name}' as about_name FROM tr_esg.wrds_company_info WHERE REPLACE(REPLACE(UPPER(orgname),'`',''),',','') LIKE '%{updated_name.upper()}%'").head(1)
  if (df_to_ad.shape[0] == 0):
    df_to_ad = db.raw_sql(f"SELECT *,'{og_name}' as about_name FROM tr_esg.wrds_company_info WHERE REPLACE(UPPER(orgname),'''','') LIKE '%{updated_name.upper()}%'").head(1)
  if (df_to_ad.shape[0] == 0):
    df_to_ad = db.raw_sql(f"SELECT *,'{og_name}' as about_name FROM tr_esg.wrds_company_info WHERE UPPER(orgname) LIKE '%{updated_second.upper()}%'").head(1)
  if (df_to_ad.shape[0] == 0):
    df_to_ad = db.raw_sql(f"SELECT *,'{og_name}' as about_name FROM tr_esg.wrds_company_info WHERE UPPER(orgname) LIKE '%{updated_2quarter.upper()}%'").head(1)
  if (df_to_ad.shape[0] == 0):
    df_to_ad = db.raw_sql(f"SELECT *,'{og_name}' as about_name FROM tr_esg.wrds_company_info WHERE UPPER(orgname) LIKE '%{updated_2half.upper()}%'").head(1)
  if (df_to_ad.shape[0] == 0):
    df_to_ad = db.raw_sql(f"SELECT *,'{og_name}' as about_name FROM tr_esg.wrds_company_info WHERE UPPER(orgname) LIKE '%{updated_2half.upper()} %'").head(1)
  if (df_to_ad.shape[0] == 0):
    df_to_ad = db.raw_sql(f"SELECT *,'{og_name}' as about_name FROM tr_esg.wrds_company_info WHERE UPPER(orgname) LIKE '%{updated_third.upper()}%'").head(1)
  dd = dd.append(df_to_ad)

print(dd.shape[0])

dd[['orgname']].to_csv('two_hundred.csv')

469


In [ ]:
name_to_orgid = dd[['orgname', 'orgid']]
name_to_orgid

In [ ]:
company_score_per_category= db.raw_sql(f"SELECT * FROM tr_esg.wrds_esg_catscores WHERE fisyear=2020")

In [ ]:
name_to_orgid.merge(company_score_per_category, how='left', left_on='orgid', right_on='orgid', indicator=True)

,orgname,orgid,fisyear,cdflag,feeddate,fyenddate,cg_bd_bf,cg_bd_bs,cg_bd_cp,cg_in_vs,cg_sh_sr,ec_ma_pe,ec_pr_sl,ec_re_cl,en_en_er,en_en_pi,en_en_rr,so_cu_pr,so_so_co,so_so_hr,so_wo_do,so_wo_eq,so_wo_hs,so_wo_td,score,cscore,controversiesscore,resourceusescore,emissionsscore,innovationscore,workforcescore,humanrightsscore,communityscore,productrespscore,managementscore,shareholdersscore,csrstrategyscore,_merge
0,Walmart Inc,32947.0,2020.0,500.0,2021-09-12,2020-01-31,0.886623,0.906218,0.380313,0.796719,0.333092,0.135473,0.895344,0.260191,0.915963,0.920998,0.943729,0.782165,0.260851,0.887719,0.807082,0.039979,0.360740,0.881856,0.845700,0.429993,0.014286,0.927184,0.771028,0.771930,0.917969,0.928571,0.972656,0.815789,0.856493,0.560383,0.718545,both
1,"Amazon.com, Inc.",114492.0,2020.0,500.0,2021-09-12,2020-12-31,0.869738,0.904743,0.806548,0.832386,0.902421,0.119645,0.051528,0.264124,0.928547,0.920998,0.898678,0.170254,0.527054,0.780654,0.744795,0.040040,0.135057,0.848896,0.864718,0.447065,0.029412,0.784722,0.977612,0.784091,0.914773,0.954545,0.903409,0.896104,0.855238,0.823871,0.608727,both
2,Apple Inc,19775.0,2020.0,500.0,2021-09-12,2020-09-26,0.676142,0.878849,0.468761,0.895881,0.854137,0.128579,0.064180,0.323031,0.919721,0.831795,0.880692,0.129399,0.553834,0.629236,0.877535,0.040188,0.850068,0.862175,0.725013,0.375665,0.026316,0.876623,0.854430,0.377049,0.810345,0.808219,0.729885,0.607143,0.825125,0.773996,0.790182,both
3,CVS Health Corp,27445.0,2020.0,500.0,2021-09-12,2020-12-31,0.750123,0.890279,0.221364,0.820186,0.914876,0.705639,0.109768,0.911021,0.921595,0.897814,0.943208,0.505187,0.987170,0.887719,0.908851,0.059883,0.952959,0.879685,0.874543,0.491619,0.108696,0.994048,0.981013,0.500000,0.928571,0.903509,0.966165,0.880165,0.783093,0.761136,0.922909,both
4,Exxon Mobil Corporation,18215.0,2020.0,510.0,2021-09-12,None,0.214224,0.269807,0.420322,0.920032,0.021636,0.171373,0.189167,0.064513,0.918828,0.795904,0.802589,0.582347,0.289163,0.656745,0.822735,0.040066,0.957306,0.786656,0.697857,0.417477,0.137097,0.927230,0.850211,0.806667,0.737154,0.418239,0.950593,0.971491,0.402917,0.377196,0.858545,both
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
464,Crown Castle International Corp,114398634.0,2020.0,510.0,2021-09-12,None,0.904794,0.869555,0.670797,0.800785,0.321281,0.820206,0.630965,0.653507,0.772061,0.140032,0.806941,0.226519,0.683676,0.858629,0.320754,0.337971,0.942544,0.846063,0.686121,0.618061,0.550000,0.831148,0.833333,0.241706,0.671543,0.361413,0.577128,0.408960,0.712829,0.737923,0.718545,both
465,Simon Property Group Inc,63242.0,2020.0,500.0,2021-09-12,2020-12-31,0.741601,0.690796,0.521668,0.933133,0.370163,0.497888,0.140918,0.812751,0.915650,0.140618,0.693989,0.514492,0.794392,0.466516,0.828807,0.041272,0.318724,0.810050,0.604118,0.604118,1.000000,0.857377,0.881877,0.241706,0.716755,0.168478,0.934840,0.408960,0.073243,0.708438,0.858545,both
466,Cerner Corporation,21451.0,2020.0,500.0,2021-09-12,2020-12-31,0.725627,0.842732,0.325964,0.737502,0.403264,0.089669,0.756113,0.876685,0.192123,0.219834,0.767476,0.247569,0.557230,0.869520,0.849264,0.279905,0.351283,0.747799,0.405971,0.405971,1.000000,0.450355,0.142308,0.000000,0.679283,0.246429,0.766932,0.136364,0.492629,0.520859,0.385818,both
467,Post Holdings Inc,111181321.0,2020.0,500.0,2021-09-12,2020-09-30,0.412062,0.332095,0.860563,0.819825,0.717286,0.844563,0.430906,0.280565,0.552137,0.403496,0.621975,0.605132,0.128932,0.819637,0.912430,0.143073,0.198125,0.753338,0.344219,0.344219,0.696078,0.356089,0.214801,0.239130,0.412621,0.319383,0.585761,0.281250,0.273369,0.474435,0.533455,both


In [ ]:
names_in_wrds_df[names_in_wrds_df['orgname'].str.contains('Chevron')]



/























/



In [ ]:
!pip3 install fuzzywuzzy

In [ ]:
import fuzzywuzzy.process as fwp

choices = list(companies_all_df.name)

def fmatch(row): 
    minscore=90 #or whatever score works for you
    try:
      choice,score = fwp.extractOne(row.orgname,choices)
    except TypeError:
      return ""
    if (score > minscore):
      print(row.orgname, choice)
    return choice if score > minscore else None

names_in_wrds_df['cmp_name'] = names_in_wrds_df.apply(fmatch,axis=1)

# merged = pd.merge(df1, 
#                   df2,
#                   left_on='df2_name',
#                   right_on='name',
#                   suffixes=['_df1','_df2'],
#                   how = 'outer') # assuming you want to keep unmatched records

In [ ]:
names_in_wrds_df=names_in_wrds_df.dropna(subset=['orgname'])

In [ ]:
names_in_wrds_df

In [ ]:
companies_all_df[companies_all_df['ticker']=='GOOGL']

In [ ]:
lll = names_in_wrds_df.merge(companies_all_df, left_on='tick', right_on='ticker', how='right', indicator=True)

In [ ]:
lll.to_csv('TOT.csv')

In [ ]:
ll_df = db.raw_sql("SELECT * FROM tr_esg.wrds_company_info WHERE tick='GOOG'")
ll_df

In [ ]:
# Specify that all figures should be displayed with 2 decimal pla
pd.options.display.float_format = '{:.2f}'.format

In [ ]:
# # Define a function that prints the percentage of missing observations in each column in descending order
def get_missing_value_df_annual(df):
  percent_missing_ideal = df.isna().sum() / (4147*11)
  missing_value_df = pd.DataFrame({'column_name': df.columns,
                                 'non_null_observations': df.count(),
                                 'missing_observations': df.isna().sum(),
                                 'percent_missing_ideal': percent_missing_ideal})
  missing_value_df.sort_values('percent_missing_ideal', ascending=False, inplace=True)
  return missing_value_df.reset_index(drop=True)

# Define a function that prints the percentage of missing observations in each column in descending order
def get_missing_value_df(df):
  # Print the no. of ISINs for which there is
  num_isins = len(df['isin'].unique())
  isin_percent = (num_isins / 4147) * 100
  print('Of all the 4147 ISINs, there is data present for ' + str(num_isins) + ' ISINs (' + str(round(isin_percent, 2)) + '%).') 
  
  # Missingness by Year for Each Variable
  print('\n')
  print('Missingness by Year for Each Variable')
  missingness_by_year_df = df.drop(columns=['isin', 'orgid', 'orgname', 'fiscal_year']).isnull().groupby(df['fiscal_year'].astype(int)).sum() / 4147 * 100
  display(missingness_by_year_df)

  # Missingness in an "ideal world" (If there was data for all 4147 ISINs by quarter for all 10 years) 
  print('\n')
  print('Missingness in an "ideal world" (If there was data for all 4147 ISINs by quarter for all 10 years)')
  percent_missing_ideal = df.isna().sum() * 100 / (4147*11)  
  missing_value_df = pd.DataFrame({'non_null_observations': df.count(),
                                 'missing_observations': df.isna().sum(),
                                 'percent_missing_ideal': percent_missing_ideal})
  missing_value_df.sort_values('percent_missing_ideal', ascending=False).reset_index(inplace=True)
  display(missing_value_df)

## Refinitiv (Updated 7/10)

Refinitiv's ESG database doesn't use the ESG as the identifier for each security. Instead, they use a propretiary `orgid` (organization id) to identify each security. Thus, we need to first identify which ISINs are contained within Refinitiv's database, and identify what sort of mapping exists between ISINs and Refinitiv's `orgid` (whether the mapping is one-to-one or one-to-many). 

### Identifying which ISINs are represented in Refinitiv's data libraries (Updated 7/10)





Let's first identity which ISINs are represented in Refinitiv's data library. 

In [ ]:
isin_list = pd.read_csv("/content/mnt/MyDrive/isin_list.txt", header = None)

start_date = datetime.date(2010, 1, 1)
end_date = datetime.date(2020, 12, 31)

vars = {'orgs': tuple(isin_list[0].unique()), 'start': start_date, 'end': end_date}

In [ ]:
refinitiv_isins = db.raw_sql("SELECT * "
                             "FROM tr_esg.wrds_company_info "
                             "WHERE orgname LIKE '%Walmart%' ")

In [ ]:
refinitiv_isins

Below, we see that some isins map to multiple organization IDs. Let's verify whether these multiple `orgid` attributes are duplicates or actually correspond to different securities. 

In [ ]:
grouped_refinitiv_isins = refinitiv_isins.groupby(by='isin').agg('count')
grouped_refinitiv_isins[grouped_refinitiv_isins['orgid'] != 1].sort_values(by='orgid', ascending=False).head()

We see below that some (`isin`, `orgid`) pairs are duplicated.

In [ ]:
duplicated_isins = grouped_refinitiv_isins[grouped_refinitiv_isins['orgid'] != 1].reset_index()['isin']

In [ ]:
pd.merge(refinitiv_isins, duplicated_isins, on='isin', copy=False).sort_values(by='isin')

However, after removing duplicate (`isin`, `orgid`) pairs, we still see that the no. of ISINs and the no. of `orgid`s differs, indicating that there are some ISINs that map to multiple `orgid`s.  

In [ ]:
unique_duplicated_isins = pd.merge(refinitiv_isins, duplicated_isins, on='isin', copy=False).drop_duplicates(subset=['isin', 'orgid']).sort_values(by='isin')
unique_duplicated_isins.head()

In [ ]:
unique_duplicated_isins['isin'].unique().shape

In [ ]:
unique_duplicated_isins['orgid'].unique().shape

Indeed, we see below that there are actually 4 ISINs that each correspond to two different `orgid`s and two different financial entities. 

In [ ]:
unique_duplicated_isins.groupby('isin').agg('count').sort_values(by='orgid', ascending=False).head(n=4)

In [ ]:
unique_duplicated_isins[unique_duplicated_isins['isin'].isin(['JP3298000005',
                                                              'IE00BY7QL619',
                                                              'NL0010545661',
                                                              'CA94106B1013'])]

### Annual Employee-related data/Social Contributions (Updated 7/10)

With the above in mind, let's collect employee-related data from Refinitiv and use the `orgid` as the identifier (keeping in mind that 8 `orgids` will correspond to a duplicated ISIN). 

In [ ]:
refinitiv_vars = {'orgids': tuple(refinitiv_isins['orgid'].unique().tolist()), 
                  'start': start_date, 
                  'end': end_date}

In [ ]:
db

In [ ]:
refinitiv_emp_df = db.raw_sql("SELECT fisyear AS fiscal_year, orgid, "
          "so_wo_eq_dp016 AS salaries_wages, "
          "so_wo_td_dp021 AS training_costs, "
          "so_wo_eq_dp001_1 AS employment_benefits, "
          "so_wo_eq_dp034 AS employee_turnover, "
          "so_so_co_dp060 AS tax_fraud_controversies, "
          "so_so_co_dp074 AS corporate_responsibilities_awards, "
          "so_wo_do_dp017 AS female_employees, "
          "so_wo_do_dp019 AS female_managers, "
          "so_wo_do_dp032 AS disabled_employees, "
          "so_wo_do_dp023 AS hrc_corporate_equality_index, "
          "so_wo_eq_dp037 AS strikes, "
          "so_wo_do_dp027 AS day_care_services, "
          "so_so_co_dp075 AS lobbying_contribution_amount, "
          "so_so_co_dp035 AS political_contributions, "
          "so_so_co_dp030 AS microfinance_impact_investments, "
          "so_so_co_dp028 AS total_donations "
          "FROM tr_esg.wrds_company_so "
          "WHERE orgid = 32947.00 AND fisyear >= 2000")

In [ ]:
refinitiv_emp_df.head(n=10)

In [ ]:
refinitiv_emp_df.shape

In [ ]:
# We have data for 3676 distinct financial entities, of which 3668 correspond to distinct ISINs. 
refinitiv_emp_df['orgid'].unique().shape

In [ ]:
# Merge with ISINs
refinitiv_emp_merged_df = pd.merge(refinitiv_isins, refinitiv_emp_df, on='orgid')

In [ ]:
# Drop any rows that have the same (isin, orgid, fiscal_year) combination
refinitiv_emp_merged_df.drop_duplicates(subset=['isin', 'orgid', 'fiscal_year'], inplace=True)

There are 4147 unique ISINs in our existing data.

In [ ]:
len(vars['orgs'])

In [ ]:
len(refinitiv_emp_df)

In [ ]:
len(refinitiv_emp_merged_df['isin'].unique())

In [ ]:
refinitiv_emp_merged_df['fiscal_year'].astype(int)

In [ ]:
refinitiv_emp_merged_df.drop(columns=['isin', 'orgid', 'orgname', 'fiscal_year']).isnull().groupby(refinitiv_emp_merged_df['fiscal_year'].astype(int)).sum() / 4147 * 100

In [ ]:
# Define a function that prints the percentage of missing observations in each column in descending order
def get_missing_value_df(df):
  # Print the no. of ISINs for which there is
  num_isins = len(df['isin'].unique())
  isin_percent = (num_isins / 4147) * 100
  print('Of all the 4147 ISINs, there is data present for ' + str(num_isins) + ' ISINs (' + str(round(isin_percent, 2)) + '%).') 
  
  # Missingness by Year for Each Variable
  print('\n')
  print('Missingness by Year for Each Variable')
  missingness_by_year_df = df.drop(columns=['isin', 'orgid', 'orgname', 'fiscal_year']).isnull().groupby(df['fiscal_year'].astype(int)).sum() / 4147 * 100
  display(missingness_by_year_df)

  # Missingness in an "ideal world" (If there was data for all 4147 ISINs by quarter for all 10 years) 
  print('\n')
  print('Missingness in an "ideal world" (If there was data for all 4147 ISINs by quarter for all 10 years)')
  percent_missing_ideal = df.isna().sum() * 100 / (4147*4*10)  
  missing_value_df = pd.DataFrame({'non_null_observations': df.count(),
                                 'missing_observations': df.isna().sum(),
                                 'percent_missing_ideal': percent_missing_ideal})
  missing_value_df.sort_values('percent_missing_ideal', ascending=False).reset_index(inplace=True)
  display(missing_value_df)

In [ ]:
get_missing_value_df(refinitiv_emp_merged_df)

In [ ]:
refinitiv_emp_df.isna().sum() / (4147 * 4 * 10)

In [ ]:
refinitiv_emp_merged_df.to_excel("/content/drive/MyDrive/Refinitiv_Employee_Data_0716.xlsx")

### Annual Environmental Data (Updated 7/10)

The following variables were collected from the `wrds_company_en` table in Refinitiv's WRDS library.        
https://wrds-www.wharton.upenn.edu/data-dictionary/tr_esg/wrds_company_en/                 



| Variable in Refinitiv Database | Description |
|:---:|:---:|
| orgid | Organization ID |
| fisyear | Fiscal Year |
| en_en_er_dp023 | CO2 Equivalents Emission Total |
| en_en_er_dp024 | CO2 Equivalents Emission Direct |
| en_en_er_dp045 | Waste Total |
| en_en_er_dp052 | Waste Recycled Total |
| en_en_er_dp056 | Hazardous Waste |
| en_en_er_dp062 | Waste Reduction Initiatives |
| en_en_er_dp058 | Water Pollutant Emissions |
| en_en_er_dp098 | Waste Recycling Ratio  |
| en_en_pi_dp027 | Fleet Fuel Consumption |
| en_en_pi_dp029 | Fleet CO2 Emissions |
| en_en_er_dp034 | NOx Emissions |
| en_en_er_dp035 | SOx Emissions |
| en_en_er_dp040 | VOC Emissions |
| en_en_er_dp032 | Ozone-Depleting Substances |
| en_en_er_dp033 | NOx and SOx Emissions Reduction Initiatives |
| en_en_er_dp036 | VOC Emissions Reduction Initiatives |
| en_en_er_dp037 | Particulate Matter Reduction Initiatives |
| en_en_rr_dp033 | Energy Use Total |
| en_en_rr_dp056 | Water Recycled |
| en_en_er_dp123 | Estimated CO2 Equivalents Emission Total |
| en_en_rr_dp063 | Natural Gas Produced (raw material in million m3) |
| en_en_rr_dp064 | Oil Produced (raw material in barrels) |
| en_en_rr_dp062 | Coal Produced (raw material in tonnes) |
| en_en_er_dp085 | Volume of Accidental Spills |
| en_en_er_dp068 | Emissions Trading |
| en_en_er_dp097 | Carbon Offsets/Credits |

In [ ]:
refinitiv_env_df = db.raw_sql("SELECT orgid, fisyear, "
"en_en_er_dp023, "
"en_en_er_dp024, "
"en_en_er_dp045, "
"en_en_er_dp052, "
"en_en_er_dp056, "
"en_en_er_dp062, "
"en_en_er_dp058, "
"en_en_er_dp098, "
"en_en_pi_dp027, "
"en_en_pi_dp029, "
"en_en_er_dp034, "
"en_en_er_dp035, "
"en_en_er_dp040, "
"en_en_er_dp032, "
"en_en_er_dp033, "
"en_en_er_dp036, "
"en_en_er_dp037, "
"en_en_rr_dp033, "
"en_en_rr_dp056, "
"en_en_er_dp123, "
"en_en_rr_dp063, "
"en_en_rr_dp064, "
"en_en_rr_dp062, "
"en_en_er_dp085, "
"en_en_er_dp068, "
"en_en_er_dp097 "
"FROM tr_esg.wrds_company_en "
"WHERE orgid IN %(orgids)s AND fisyear >= 2010 AND fisyear <= 2020",
params=refinitiv_vars)

In [ ]:
refinitiv_env_merged_df = pd.merge(refinitiv_isins, refinitiv_env_df, on='orgid')

In [ ]:
refinitiv_env_merged_df.drop_duplicates(subset=['isin', 'orgid', 'fisyear'], inplace=True)

In [ ]:
refinitiv_env_merged_df.rename(columns={
"fisyear": "fiscal_year",    
"en_en_er_dp023": "CO2 Equivalents Emission Total",
"en_en_er_dp024": "CO2 Equivalents Emission Direct",
"en_en_er_dp045": "Waste Total",
"en_en_er_dp052": "Waste Recycled Total",
"en_en_er_dp056": "Hazardous Waste",
"en_en_er_dp062": "Waste Reduction Initiatives",
"en_en_er_dp058": "Water Pollutant Emissions",
"en_en_er_dp098": "Waste Recycling Ratio",
"en_en_pi_dp027": "Fleet Fuel Consumption",
"en_en_pi_dp029": "Fleet CO2 Emissions",
"en_en_er_dp034": "NOx Emissions",
"en_en_er_dp035": "SOx Emissions",
"en_en_er_dp040": "VOC Emissions",
"en_en_er_dp032": "Ozone-Depleting Substances",
"en_en_er_dp033": "NOx and SOx Emissions Reduction Initiatives",
"en_en_er_dp036": "VOC Emissions Reduction Initiatives",
"en_en_er_dp037": "Particulate Matter Reduction Initiatives",
"en_en_rr_dp033": "Energy Use Total",
"en_en_rr_dp056": "Water Recycled",
"en_en_er_dp123": "Estimated CO2 Equivalents Emission Total",
"en_en_rr_dp063": "Natural Gas Produced (raw material in million m3)",
"en_en_rr_dp064": "Oil Produced (raw material in barrels)",
"en_en_rr_dp062": "Coal Produced (raw material in tonnes)",
"en_en_er_dp085": "Volume of Accidental Spills",
"en_en_er_dp068": "Emissions Trading",
"en_en_er_dp097": "Carbon Offsets/Credits"},
inplace=True)

In [ ]:
refinitiv_env_merged_df['Energy Use Total'].value_counts()

In [ ]:
get_missing_value_df(refinitiv_env_merged_df)

In [ ]:
refinitiv_env_merged_df.to_excel("/content/drive/MyDrive/Refinitiv_Environmental_Data_0716.xlsx")

### Legacy code for Refinitiv Employee Data (**Deprecated**)

In [ ]:
# Import Environmental Data that we downloaded from Refinitiv via the WRDS web UI
refinitiv_env_df = pd.read_excel("/content/mnt/MyDrive/Refinitiv_environmental_data.xlsx")

In [ ]:
refinitiv_env_df.columns

In [ ]:
# List of Refinitiv Organization IDs corresponding to the desired ISINs
orgids = tuple(refinitiv_env_df['Organization ID'].unique().tolist())

In [ ]:
# Company specific data from Refinitiv
parm = {'orgs': orgids}

# Refinitiv Employee Metric Dataframe
refinitiv_emp_df = db.raw_sql("SELECT fisyear, orgid, "
          "so_wo_eq_dp016 AS salaries_wages, "
          "so_wo_td_dp021 AS training_costs, "
          "so_wo_eq_dp001_1 AS employment_benefits, "
          "so_so_co_dp035 AS employee_turnover, "
          "so_so_co_dp035 AS political_contributions, "
          "so_so_co_dp060 AS tax_fraud_controversies "
          "FROM tr_esg.wrds_company_so "
          "WHERE orgid IN %(orgs)s AND fisyear >= 2010",
           params=parm)
refinitiv_emp_df.head(n=10)

In [ ]:
refinitiv_emp_df.iloc[:,2:].describe()

In [ ]:
refinitiv_emp_missing_df = get_missing_value_df(refinitiv_emp_df)
refinitiv_emp_missing_df

### Refinitiv Environmental Data (**Deprecated**)


In [ ]:
refinitiv_env_df.iloc[:, 2:].describe()

In [ ]:
refinitiv_env_missing_df = get_missing_value_df(refinitiv_env_df)
refinitiv_env_missing_df

## Worldscope (Updated 8/11)


In [ ]:
isin_list = pd.read_csv("/content/mnt/MyDrive/isin_list.txt", header = None)

start_date = datetime.date(2010, 1, 1)
end_date = datetime.date(2020, 12, 31)

vars = {'orgs': tuple(isin_list[0].unique()), 'start': start_date, 'end': end_date}

We first examine how many ISINs are represented in the Worldscope data. We see below that Worldscope has data for 4032 ISINs (approx. 97% of all 4147 of our desired ISINs). Like Refinitiv, Worldscope uses its propreitary ID to identify each security within its ESG-related datasets, so we need to obtain a mapping from ISINs to Worldscope's propreitary ID. 

In [ ]:
# Querying Worldscope's entity ID table
# https://wrds-www.wharton.upenn.edu/data-dictionary/tr_worldscope/wrds_ws_ids/

ws_isins_df = db.raw_sql("SELECT code, item6008 AS isin, item6035 AS ws_id, item6105 AS ws_perm_id "
"FROM wrds_ws_ids "
"WHERE item6008 IN %(orgs)s", 
params=vars)

# Checked separately that there are no duplicate ISINs in the Worldscope data
len(ws_isins_df['isin'].unique())

### Quarterly Financial Data (Taxes, Payables, EPS, CAPEX) (Updated 7/11)

The following variables were collected from Worldscope's Quarterly Fundamentals dataset.             
https://wrds-www.wharton.upenn.edu/data-dictionary/tr_worldscope/wrds_ws_fundq/                   

| Variable in Worldscope database | Description |
|:---:|:---:|
| item6008 | ISIN |
| item5601 | Ticker |
| date_ | Date (Quarter) |
| item1451 | Income Taxes |
| item3063 | Income Taxes Payable |
| item3263 | Deferred Taxes |
| item3040 | Accounts Payable |
| item3255 | Total Debt |
| item8231 | Total Debt % Common Equity |
| item8236 | Total Debt % Total Assets |
| item3251 | Long Term Debt |
| item8226 | Long Term Debt % Common Equity |
| item4701 | Reduction in Long Term Debt |
| item4401 | Long Term Borrowings |
| item8106 | Current Ratio |
| item4601 | Capital Expenditures (additions to fixed assets) |
| item8416 | Capital Expenditures % Total Assets |
| item8421 | Capital Expenditures % Total Sales |
| item18173 | Unconsolidated/Parent Company Earnings Per Share |
| item8601 | Earnings Per Share - 1 Yr Annual Growth |
| item8699 | Earnings Per Share - Quarter |

In [ ]:
ws_financial_df = db.raw_sql("SELECT item6008, "
"item5601, "
"date_, "
"year_, "
"item6099, "
"item1451, "
"item3063, "
"item3263, "
"item3040, "
"item3255, "
"item8231, "
"item8236, "
"item3251, "
"item8226, "
"item4701, "
"item4401, "
"item8106, "
"item4601, "
"item8416, "
"item8421, "
"item18173, "
"item8601, "
"item8699, "
"item11556, "
"item11557, "
"item11558, "
"item11559 "
"FROM wrds_ws_funda "
"WHERE item6008 IN %(orgs)s AND date_ >= %(start)s AND date_ <= %(end)s",
params=vars)

In [ ]:
ws_financial_df.rename(columns={
    "item6008": "ISIN",
    "item5601": "Ticker", 
    "date_": "Date (Quarter)", 
    "year_": "Year",
    "item6099": "Currency", 
    "item1451": "Income Taxes",
    "item3063": "Income Taxes Payable", 
    "item3263": "Deferred Taxes",
    "item3040": "Accounts Payable", 
    "item3255": "Total Debt", 
    "item8231": "Total Debt % Common Equity", 
    "item8236": "Total Debt % Total Assets", 
    "item3251": "Long Term Debt", 
    "item8226": "Long Term Debt % Common Equity", 
    "item4701": "Reduction in Long Term Debt", 
    "item4401": "Long Term Borrowings", 
    "item8106": "Current Ratio",
    "item4601": "Capital Expenditures (additions to fixed assets)", 
    "item8416": "Capital Expenditures % Total Assets", 
    "item8421": "Capital Expenditures % Total Sales", 
    "item18173": "Unconsolidated/Parent Company Earnings Per Share", 
    "item8601": "Earnings Per Share - 1 Yr Annual Growth", 
    "item8699": "Earnings Per Share - Quarter",
    "item11556": "Restated Data Exists",
    "item11557": "Restatement Type Indicator",
    "item11558": "Early Release Indicator",
    "item11559": "Restatement Reason Code"
    },
    inplace=True)

In [ ]:
ws_financial_df = ws_financial_df.astype({"Year": "int"})

In [ ]:
ws_financial_df.drop_duplicates(inplace=True)

In [ ]:
ws_financial_df['Calendar Year'] = pd.DatetimeIndex(ws_financial_df['Date (Quarter)']).year

In [ ]:
ws_financial_df.rename(columns={"Year": "Fiscal Year"}, inplace=True)

In [ ]:
ws_financial_df.head(n=20)

In [ ]:
ws_financial_annual_availability_df = ws_financial_df[['ISIN', 'Calendar Year', 'Date (Quarter)']].groupby(by='ISIN').nunique()

We see that the majority of the ISINs (around 76%) had financial data for all 10 years. Around 67% of the ISINs had data for all 44 quarters during 2010-2020.

In [ ]:
ws_financial_annual_availability_df.hist(column='Calendar Year', density=True)
plt.title('Distribution of the No. of Years per ISIN \n(WorldScope Financial Data)')
plt.xlabel('No. of Years')
plt.ylabel('Percentage of ISINs in WorldScope')
plt.xticks(np.arange(start=0, stop=12, step=1))
plt.show()

In [ ]:
ws_financial_annual_availability_df.hist(column='Date (Quarter)', density=True, bins = 45)
plt.title('Distribution of the No. of Quarters per ISIN \n(WorldScope Financial Data)')
plt.xlabel('No. of Quarters')
plt.ylabel('Percentage of ISINs in WorldScope')
plt.xticks(np.arange(start=0, stop=47, step=5))
plt.show()

In [ ]:
# Summary statistics for the no. of years/quarters during which data was available for each ISIN in the WorldScope dataset
ws_financial_annual_availability_df.describe()

In [ ]:
ws_financial_df.groupby(by=["ISIN", "Year"]).agg("count").head(n=10)

In [ ]:
# Define a function that prints the percentage of missing observations in each column in descending order
def ws_get_missing_value_df(df):
  # Print the no. of ISINs for which there is data
  num_isins = len(df['ISIN'].unique())
  isin_percent = (num_isins / 4147) * 100
  print('Of all the 4147 ISINs, there is data present for ' + str(num_isins) + ' ISINs (' + str(round(isin_percent, 2)) + '%).') 
  
  # Missingness by Year for Each Variable
  print('\n')
  print('Missingness by Year for Each Variable')
  missingness_by_year_df = (df.drop(columns=['ISIN', 'Calendar Year', 'Date (Quarter)', 'Fiscal Year', 'Ticker']).isnull().groupby(df['Calendar Year'].astype(int)).sum() / (4147*4))*100
  display(missingness_by_year_df)

  # Missingness in an "ideal world" (If there was data for all 4147 ISINs by quarter for all 11 years) 
  print('\n')
  print('Missingness in an "ideal world" (If there was data for all 4147 ISINs by quarter for all 11 years)')
  percent_missing_ideal = (df.drop(columns=['ISIN', 'Calendar Year', 'Date (Quarter)', 'Fiscal Year', 'Ticker']).isna().sum() / (4147*4*11))*100
  missing_value_df = pd.DataFrame({'percent_missing_ideal': percent_missing_ideal})
  missing_value_df.rename(columns={'percent_missing_ideal': '% Missing Data (Out of All ISINs for All Years)'}, inplace=True)
  missing_value_df.sort_values(by='% Missing Data (Out of All ISINs for All Years)', inplace=True)
  display(missing_value_df)

ws_get_missing_value_df(ws_financial_df)

In [ ]:
ws_financial_df.to_excel("/content/drive/MyDrive/WorldScope_FinanciL_Data_with_Currency_0716.xlsx")

### Annual Financial Data (Updated 8/11)

Following our meeting with FCLT on 8/9, we will now be collecting annual versions of the Worldscope financial variables listed above. The variables are collected from the `wrds_ws_funda` WRDS table:      
https://wrds-www.wharton.upenn.edu/data-dictionary/tr_worldscope/wrds_ws_funda/ 

All the variables in this table are identical to the `wrds_ws_fundq` where we previously obtained quarterly variables (with the exception of Quarter-specific variables such as quarterly earnings per share). 

In [ ]:
# item5350 is fiscal period end date
ws_financial_df = db.raw_sql("SELECT item6008, "
"item5601, "
"item5350, "
"year_, "
"item11556, "
"item11557, "
"item11558, "
"item11559, "
"item7536, "
"item6099, "
"item1451, "
"item3063, "
"item3263, "
"item3040, "
"item3255, "
"item8231, "
"item8236, "
"item3251, "
"item8226, "
"item4701, "
"item4401, "
"item8106, "
"item4601, "
"item8416, "
"item8421, "
"item18173, "
"item8601 "
"FROM wrds_ws_funda "
"WHERE item6008 IN %(orgs)s AND item5350 >= %(start)s AND item5350 <= %(end)s",
params=vars)

In [ ]:
# Rename columns
ws_financial_df.rename(columns={
    "item6008": "ISIN",
    "item5601": "Ticker", 
    "item5350": "Fiscal Period End Date", 
    "year_": "Year",
    "item6099": "Currency", 
    "item1451": "Income Taxes",
    "item3063": "Income Taxes Payable", 
    "item3263": "Deferred Taxes",
    "item3040": "Accounts Payable", 
    "item3255": "Total Debt", 
    "item8231": "Total Debt % Common Equity", 
    "item8236": "Total Debt % Total Assets", 
    "item3251": "Long Term Debt", 
    "item8226": "Long Term Debt % Common Equity", 
    "item4701": "Reduction in Long Term Debt", 
    "item4401": "Long Term Borrowings", 
    "item8106": "Current Ratio",
    "item4601": "Capital Expenditures (additions to fixed assets)", 
    "item8416": "Capital Expenditures % Total Assets", 
    "item8421": "Capital Expenditures % Total Sales", 
    "item18173": "Unconsolidated/Parent Company Earnings Per Share", 
    "item8601": "Earnings Per Share - 1 Yr Annual Growth",
    "item11556": "restated_data_exists",
    "item11557": "restatement_type_indicator",
    "item11558": "early_release_indicator",
    "item11559": "restatement_reason_code",
    "item7536": "accounting_standards_followed"
    },
    inplace=True)

In [ ]:
ws_financial_df.drop_duplicates(inplace=True)

In [ ]:
ws_financial_df.head()

In [ ]:
# TODO: filter all rows where "Restatement Type Indicator" == 1 (indicating a restatement and not a revision)
!pip3 install pandasql

In [ ]:
from pandasql import sqldf

In [ ]:
ws_financial_df['restatement_reason_code'].value_counts()

In [ ]:
ws_financial_df.astype({'Year': 'int'}, copy=False)

In [ ]:
ws_financial_df['keep_column'] = [True] * len(ws_financial_df)

In [ ]:
ws_financial_df[ws_financial_df['ISIN'] == 'GB00B0SWJX34'].index[0]

In [ ]:
list(ws_financial_df_copy.index)[-1]

In [ ]:
ws_financial_df_copy = ws_financial_df.copy(deep=True)
ws_financial_df_copy.reset_index(inplace=True)
worldscope_isins = ws_financial_df_copy['ISIN'].unique()
ncols = len(ws_financial_df_copy.columns)
for isin in worldscope_isins:
  isin_df = ws_financial_df_copy[ws_financial_df_copy['ISIN'] == isin]
  years = isin_df['Year'].unique()
  for year in years:
    year_df = isin_df[isin_df['Year'] == year]
    for row in year_df.itertuples():
      # print(row.restated_data_exists)
      # if row.restated_data_exists == 1:
        if row.restatement_reason_code == None or row.early_release_indicator == 2:
          # row = ws_financial_df_copy.loc[(ws_financial_df_copy['ISIN'] == isin) & (ws_financial_df_copy['year'] == year) & (ws_financial_df_copy['restated_data_exists'] == 1)].index[0]
          # print('isin ' + str(isin))
          # print('year ' + str(year))
          # print(row.Index)
          ws_financial_df_copy.iat[row.Index, ncols - 1] = False
        


# def handle_restatement(grouped_df):
#   print(grouped_df)


# ws_financial_df.groupby(by=['ISIN', 'Year']).apply(handle_restatement)


# for isin in worldscope_isins:
#   isin_df = ws_financial_df[ws_financial_df['ISIN'] == isin]
#   isin_df.group

In [ ]:
ws_financial_df_copy['keep_column'].value_counts()

In [ ]:
ws_financial_df_copy[ws_financial_df_copy['keep_column'] == True].groupby(by=['ISIN']).agg('count').sort_values(by='Year', ascending=False)

In [ ]:
ws_financial_output_df = ws_financial_df_copy[ws_financial_df_copy['keep_column'] == True].drop(columns=['index', 'restated_data_exists', 'keep_column'])
ws_financial_output_df

In [ ]:
ws_financial_df_copy[ws_financial_df_copy['ISIN'] == 'CNE000000CG9'].sort_values(by='Year', ascending=False)

In [ ]:
ws_financial_df_copy.shape

In [ ]:
query = '''
SELECT *
FROM ws_financial_df
WHERE (restated_data_exists = 1 OR restatement_type_indicator = 1)
ORDER BY Year
'''
sqldf(query)

# Restatament logic
if restatement_type_indicator == 1:
  if restatement_reason_code == None or early_release_indicator == None:
    keep original row, delete the restatement row 
  else if restatement_reason_code != None or early_release_indicator == 2
    delete original row, keep restatement row



# TODO: delete the restated_data_exists column after everything ends???

# (can ignore rows where restated_data_exists == 2)

In [ ]:
ws_financial_df.sort_values(by=['ISIN', 'Year']).head(n=10)

In [ ]:
# Define a function that prints the percentage of missing observations in each column in descending order
def ws_get_missing_value_df(df):
  # Print the no. of ISINs for which there is data
  num_isins = len(df['ISIN'].unique())
  isin_percent = (num_isins / 4147) * 100
  print('Of all the 4147 ISINs, there is data present for ' + str(num_isins) + ' ISINs (' + str(round(isin_percent, 2)) + '%).') 
  
  # Missingness by Year for Each Variable
  print('\n')
  print('Missingness by Year for Each Variable')
  missingness_by_year_df = (df.drop(columns=['ISIN', 'Year', 'Fiscal Period End Date', 'Ticker']).isnull().groupby(df['Year'].astype(int)).sum() / (4147*11))*100
  display(missingness_by_year_df)

  # Missingness in an "ideal world" (If there was data for all 4147 ISINs by quarter for all 11 years) 
  print('\n')
  print('Missingness in an "ideal world" (If there was data for all 4147 ISINs for all 11 years)')
  percent_missing_ideal = (df.drop(columns=['ISIN', 'Year', 'Fiscal Period End Date', 'Ticker']).isna().sum() / (4147*11))*100
  missing_value_df = pd.DataFrame({'percent_missing_ideal': percent_missing_ideal})
  missing_value_df.rename(columns={'percent_missing_ideal': '% Missing Data (Out of All ISINs for All Years)'}, inplace=True)
  missing_value_df.sort_values(by='% Missing Data (Out of All ISINs for All Years)', inplace=True)
  display(missing_value_df)

ws_get_missing_value_df(ws_financial_df)

### Annual Financial Data with Restatements (Updated 8/19)



**11559 - Restatement Reason Code**

Provides standardized coding for identifying the general reason restated accounts have been provided. Only one code is provided. If a company adopted a Change in GAAP followed and also discontinued operations, a code would be given for Change in GAAP followed together with other events.      
1   Where the accounts are restated due to Discontinued Operations, Spin-offs or de-mergers.               
2   Where the accounts are restated due to the adoption of a new accounting policy or policies related to specific item or items               
3   Where the accounts are restated due to Acquisition/Mergers               
4   Where the accounts are restated due to a change in accounting GAAP followed by the company               
5   Where the accounts are restated due to change in accounting GAAP              followed together with other events
6   Where the accounts are restated due to multiple events             
7   Where the accounts are restated due to reasons other than those noted above             
8   Where the accounts are reclassified.             
Existing field 07536 (accounting standards followed) can be used to identify the GAAP standard followed. Code 23 identifies if a company has adopted IFRS.
Example: If 11559 =4 and 07536 = 23, the Restatement is for a company which has adopted IFRS.
Reason code 8 was introduced to identify restated comparative accounts due to a reclassification, i.e. where the presentation of an item is changed or accounting concepts are being classified differently. Data for this field is generally available from 2009-2010 onwards.
A company can also issue restated data to correct for accounting irregularities. This is when a company has made a material error in the application of existing accounting standards or even has committed fraud. The Worldscope treatment of accounting irregularities remains the same: the figures will be amended within the original time series.

### Annnual Pensions Data (Updated 7/11)

The following pensions-related data (available annually) were collected from the table `wrds_ws_pension`.        
https://wrds-www.wharton.upenn.edu/data-dictionary/tr_worldscope/wrds_ws_pension/           

| Variable in Worldscope database | Description |
|:---:|:---:|
| item6008 | ISIN |
| item5601 | Ticker |
| year_ | Fiscal Year |
| item18806 | Pension Discount Rate |
| item18811 | Pension Service Costs |
| item18813 | Pension Interest Costs |
| item18822 | Pension Net Interest Costs |
| item18821 | Net Pension Liability |
| item18810 | Net Periodic Pension Expenses |
| item18831 | Expected Long-Term Return Rate on Pension Assets |
| item18808 | Accumulated Benefit Obligation |
| item18809 | Projected Benefit Obligation |

In [ ]:
ws_pensions_df = db.raw_sql("SELECT item6008, "
"item5601, "
"year_, "
"item18806, "
"item18811, "
"item18813, "
"item18822, "
"item18821, "
"item18810, "
"item18831, "
"item18808, "
"item18809 "
"FROM wrds_ws_pension "
"WHERE item6008 IN %(orgs)s AND year_ >= 2010 AND year_ <= 2020",
params=vars)

In [ ]:
ws_pensions_df.rename(columns={"item6008": "ISIN", 
"item5601": "Ticker", 
"year_": "Year", 
"item18806": "Pension Discount Rate", 
"item18811": "Pension Service Costs", 
"item18813": "Pension Interest Costs", 
"item18822": "Pension Net Interest Costs", 
"item18821": "Net Pension Liability", 
"item18810": "Net Periodic Pension Expenses", 
"item18831": "Expected Long-Term Return Rate on Pension Assets", 
"item18808": "Accumulated Benefit Obligation", 
"item18809": "Projected Benefit Obligation"},
inplace=True)

In [ ]:
ws_pensions_df.drop_duplicates(inplace=True)

In [ ]:
ws_pensions_df['Accumulated Benefit Obligation'].value_counts()

In [ ]:
ws_pensions_annual_availability_df = ws_pensions_df[['ISIN', 'Year']].groupby(by='ISIN').nunique()
ws_pensions_annual_availability_df.hist(column='Year', density=True)
plt.title('Distribution of the No. of Years per ISIN \n(WorldScope Pensions Data)')
plt.xlabel('No. of Years')
plt.ylabel('Percentage of ISINs in WorldScope')
plt.xticks(np.arange(start=0, stop=12, step=1))
plt.show()

In [ ]:
def ws_pensions_get_missing_value_df(df):
  # Print the no. of ISINs for which there is data
  num_isins = len(df['ISIN'].unique())
  isin_percent = (num_isins / 4147) * 100
  print('Of all the 4147 ISINs, there is data present for ' + str(num_isins) + ' ISINs (' + str(round(isin_percent, 2)) + '%).') 

  # Missingness by Year for Each Variable
  print('\n')
  print('Missingness by Year for Each Variable')
  missingness_by_year_df = (df.drop(columns=['ISIN', 'Year', 'Ticker']).isnull().groupby(df['Year'].astype(int)).sum() / (4147*4))*100
  display(missingness_by_year_df)

  # Missingness in an "ideal world" (If there was data for all 4147 ISINs by quarter for all 11 years) 
  print('\n')
  print('Missingness in an "ideal world" (If there was data for all 4147 ISINs by quarter for all 11 years)')
  percent_missing_ideal = (df.drop(columns=['ISIN', 'Year', 'Ticker']).isna().sum() / (4147*4*11))*100
  missing_value_df = pd.DataFrame({'percent_missing_ideal': percent_missing_ideal})
  missing_value_df.rename(columns={'percent_missing_ideal': '% Missing Data (Out of All ISINs for All Years)'}, inplace=True)
  missing_value_df.sort_values(by='% Missing Data (Out of All ISINs for All Years)', inplace=True)
  display(missing_value_df)

ws_pensions_get_missing_value_df(ws_pensions_df)

### Exporting Worldscope Data (Updated 7/11)

In [ ]:
# Export the Worldscope Finance & Pensions dataframes as an Excel file

with pd.ExcelWriter("/content/mnt/MyDrive/FCLT_Worldscope_Financial_Data_With_Restatements_0821.xlsx") as writer:
  ws_financial_output_df.to_excel(writer, "Financial Data with Restatements", index=False)

In [ ]:
# Export the Worldscope Finance & Pensions dataframes as an Excel file

with pd.ExcelWriter("/content/drive/MyDrive/FCLT_Worldscope_Data_0711.xlsx") as writer:
  ws_financial_df.to_excel(writer, "Quarterly Financial Data (Taxes, Payables, EPS, CAPEX) (Worldscope)")
  ws_pensions_df.to_excel(writer, "Annual Pensions Data (Worldscope)")

## Factset (Updated 7/11)

### Importing ISIN & other date variables

***ff_advanced_der_af_am***:
(**derived** items from balance sheet etc.)

ff_inc_tax_curr	(double) Income Tax - Current - Total          
ff_inc_tax_dfd	(double) Deferred Income Taxes          
ff_tax_rate	(double) Tax Rate          

***wrds_ffheader_usc_v3*** (Factset Fundamentals V3 - Header Identifiers)      

Mapping from a company's ISIN to Factset's propreitary `fsym_id` (identifier for companies)

In [ ]:
isin_df = pd.read_csv('/content/drive/MyDrive/isin_list.txt', header = None)

In [ ]:
start_date = datetime.date(2010, 1, 1)
end_date = datetime.date(2020, 12, 31)

vars = {'isins': tuple(isin_df[0].unique()), 'start': start_date, 'end': end_date}

In [ ]:
factset_isins = db.raw_sql("SELECT DISTINCT isin, fsym_id FROM wrds_ffheader_usc_v3 "
          "WHERE isin in %(isins)s", params=vars)

***wrds_fund_af_usc*** (Factset fundamental - Annual Fiscal US and CAN)             
`isin`              
`date`             
`entity_name`             

ff_pay_acct	(double) ACCOUNTS PAYABLE           
ff_pay_acct_cf	(double) INCREASE / DECREASE IN ACCOUNTS PAYABLE           
ff_pay_acct_sales	(double) ACCOUNTS PAYABLE/SALES           
ff_pay_div	(double) DIVIDENDS PAYABLE           
ff_pay_out_ratio	(double) DIVIDEND PAYOUT (percent EARNINGS) - TOTAL DOLLAR           
ff_pay_tax	(double) INCOME TAXES PAYABLE           
ff_pay_tax_cf	(double) INCREASE / DECREASE IN INCOME TAX PAYABLE           
ff_pay_tax_dfd_tax	(double) INCOME TAX PAYABLE PLUS SHORT TERM DEFERRED TAX           
ff_pay_turn_days	(double) Days of Payables Outstanding           

ff_int_pay	(double) INTEREST PAYABLE

ff_debt_st_notes_pay	(double) Notes Payable


### Quarterly Accounts Payable and other Financial Data (Updated 7/11)

#### Quarterly Data

In [ ]:
# North American (US & CAN) data

factset_payable_usc_v3_df = db.raw_sql("SELECT isin, date, proper_name, currency, "
"ff_pay_acct AS accounts_payable, "
"ff_pay_acct_sales AS accounts_payable_sales_ratio, "
"ff_dps AS payables_per_share, "
"ff_cash_div_covg_ratio AS cash_dividend_payout_ratio, "
"ff_pay_tax AS income_taxes_payable, "
"ff_tax_non_inc AS non_income_taxes, "
"ff_dfd_tax AS deferred_taxes "
"FROM wrds_fund_qf_usc_v3 "
"WHERE isin in %(isins)s AND date >= %(start)s AND date <= %(end)s",
params=vars)

In [ ]:
# International (non-North American) data
factset_payable_int_v3_df = db.raw_sql("SELECT isin, date, proper_name, currency, "
"ff_pay_acct AS accounts_payable, "
"ff_pay_acct_sales AS accounts_payable_sales_ratio, "
"ff_dps AS payables_per_share, "
"ff_cash_div_covg_ratio AS cash_dividend_payout_ratio, "
"ff_pay_tax AS income_taxes_payable, "
"ff_tax_non_inc AS non_income_taxes, "
"ff_dfd_tax AS deferred_taxes "
"FROM wrds_fund_qf_int_v3 "
"WHERE isin in %(isins)s AND date >= %(start)s AND date <= %(end)s",
params=vars)

In [ ]:
factset_payable_df = pd.concat([factset_payable_usc_v3_df, factset_payable_int_v3_df])

In [ ]:
factset_payable_df.drop_duplicates(inplace=True)

In [ ]:
factset_payable_df['year'] = pd.DatetimeIndex(factset_payable_df['date']).year

In [ ]:
factset_payable_annual_availability_df = factset_payable_df[['isin', 'year', 'date']].groupby(by='isin').nunique()
factset_payable_annual_availability_df.hist(column='year', density=True)
plt.title('Distribution of the No. of Years per ISIN \n(Factset Financial Data)')
plt.xlabel('No. of Years')
plt.ylabel('Proportion of ISINs in Factset')
plt.xticks(np.arange(start=0, stop=12, step=1))
plt.show()

In [ ]:
factset_payable_annual_availability_df.hist(column='date', density=True, bins = 45)
plt.title('Distribution of the No. of Quarters per ISIN \n(Factset Financial Data)')
plt.xlabel('No. of Quarters')
plt.ylabel('Proportion of ISINs in Factset')
plt.xticks(np.arange(start=0, stop=47, step=5))
plt.show()

In [ ]:
def factset_get_missing_value_df(df):
  # Print the no. of ISINs for which there is data
  num_isins = len(df['isin'].unique())
  isin_percent = (num_isins / 4147) * 100
  print('Of all the 4147 ISINs, there is data present for ' + str(num_isins) + ' ISINs (' + str(round(isin_percent, 2)) + '%).') 

  # Missingness by Year for Each Variable
  print('\n')
  print('Missingness by Year for Each Variable')
  missingness_by_year_df = (df.drop(columns=['isin', 'date', 'year', 'proper_name']).isnull().groupby(df['year'].astype(int)).sum() / (4147))*100
  display(missingness_by_year_df)

  # Missingness in an "ideal world" (If there was data for all 4147 ISINs by quarter for all 11 years) 
  print('\n')
  print('Missingness in an "ideal world" (If there was data for all 4147 ISINs by quarter for all 11 years)')
  percent_missing_ideal = (df.drop(columns=['isin', 'date', 'year', 'proper_name']).isna().sum() / (4147*11))*100
  missing_value_df = pd.DataFrame({'percent_missing_ideal': percent_missing_ideal})
  missing_value_df.rename(columns={'percent_missing_ideal': '% Missing Data (Out of All ISINs for All Years)'}, inplace=True)
  missing_value_df.sort_values(by='% Missing Data (Out of All ISINs for All Years)', inplace=True)
  display(missing_value_df)

# factset_get_missing_value_df(factset_payable_df)

In [ ]:
factset_payable_df.to_excel("/content/drive/MyDrive/Factset_Payable_Data_with_Currency_0716.xlsx")

#### Annual Data (Updated 8/11)

| Variable in FactSet Database          | Description                  |
|---------------------|-----------------------------------------------|
| ff_pay_acct | accounts_payable |
| ff_pay_acct_sales | accounts_payable_sales_ratio |
| ff_pay_div | dividends_payable |
| ff_pay_out_ratio | dividend_payout_ratio |
| ff_pay_tax | income_taxes_payable |
| ff_int_pay | interest_payable |

In [ ]:
factset_payable_usc_df = db.raw_sql("SELECT isin, date, entity_name,  "
"ff_pay_acct AS accounts_payable, "
"ff_pay_acct_sales AS accounts_payable_sales_ratio, "
"ff_pay_div AS dividends_payable, "
"ff_pay_out_ratio AS dividend_payout_ratio, "
"ff_pay_tax AS income_taxes_payable, "
"ff_int_pay AS interest_payable "
"FROM wrds_fund_af_usc "
"WHERE isin in %(isins)s",
params=vars)

In [ ]:
factset_payable_usc_v3_df = db.raw_sql("SELECT isin, date, proper_name,  "
"ff_pay_acct AS accounts_payable, "
"ff_pay_acct_sales AS accounts_payable_sales_ratio, "
"ff_pay_div AS dividends_payable, "
"ff_pay_out_ratio AS dividend_payout_ratio, "
"ff_pay_tax AS income_taxes_payable, "
"ff_int_pay AS interest_payable "
"FROM wrds_fund_af_usc_v3 "
"WHERE isin in %(isins)s AND date >= %(start)s AND date <= %(end)s",
params=vars)

In [ ]:
factset_payable_usc_v3_df['isin'].unique().shape

In [ ]:
factset_payable_usc_v3_df.describe()

International (non-US & CAN) data from FactSet

In [ ]:
factset_payable_int_df = db.raw_sql("SELECT isin, date, entity_name, currency, "
"ff_pay_acct AS accounts_payable, "
"ff_pay_acct_sales AS accounts_payable_sales_ratio, "
"ff_pay_div AS dividends_payable, "
"ff_pay_out_ratio AS dividend_payout_ratio, "
"ff_pay_tax AS income_taxes_payable, "
"ff_int_pay AS interest_payable "
"FROM wrds_fund_af_int "
"WHERE isin in %(isins)s",
params=vars)

In [ ]:
factset_payable_int_v3_df = db.raw_sql("SELECT isin, date, proper_name,  "
"ff_pay_acct AS accounts_payable, "
"ff_pay_acct_sales AS accounts_payable_sales_ratio, "
"ff_pay_div AS dividends_payable, "
"ff_pay_out_ratio AS dividend_payout_ratio, "
"ff_pay_tax AS income_taxes_payable, "
"ff_int_pay AS interest_payable "
"FROM wrds_fund_af_int_v3 "
"WHERE isin in %(isins)s AND date >= %(start)s AND date <= %(end)s",
params=vars)

In [ ]:
get_missing_value_df(factset_payable_int_v3_df)

In [ ]:
len(factset_payable_usc_v3_df['isin'].unique().tolist()) + len(factset_payable_int_v3_df['isin'].unique().tolist())

In [ ]:
factset_payable_df = pd.concat([factset_payable_usc_v3_df, factset_payable_int_v3_df])

1441 unique ISINs are covered through FactSet (i.e. 35% of all the ISINs). 

### *Annual* Capital Expenditure-related variables (Updated 8/11)

The following quarterly Capital Expenditure-related variables were collected from FactSet:     

| Variable in FactSet Database          | Description                  |
|---------------------|-----------------------------------------------|
| ff_capex            | Capital Expenditures (Total)                  |
| ff_capex_5ygr       | Capital Spending - 5Yr Growth Rate            |
| ff_capex_assets     | Capital Expenditure percent Total Assets      |
| ff_capex_fix        | Capital Expenditures (Fixed Assets)           |
| ff_capex_fix_assets | Capital Expediture percent Gross Fixed Assets |
| ff_capex_oth        | Capital Expenditures (Other Assets)           |
| ff_capex_ps_cf      | Capital Expenditures Per Share                |
| ff_capex_sales      | Capital Expenditure percent Total Sales       |

In [ ]:
factset_capex_int_v3_df = db.raw_sql("SELECT isin, date, proper_name, currency, ff_capex, ff_capex_5ygr, ff_capex_assets, "
"ff_capex_fix, ff_capex_oth, ff_capex_ps_cf, ff_capex_sales "
"FROM wrds_fund_af_int_v3 "
"WHERE isin in %(isins)s AND date >= %(start)s AND date <= %(end)s",
params=vars)

In [ ]:
factset_capex_usc_v3_df = db.raw_sql("SELECT isin, date, proper_name, currency, ff_capex, ff_capex_5ygr, ff_capex_assets, "
"ff_capex_fix, ff_capex_oth, ff_capex_ps_cf, ff_capex_sales "
"FROM wrds_fund_af_usc_v3 "
"WHERE isin in %(isins)s AND date >= %(start)s AND date <= %(end)s",
params=vars)

In [ ]:
factset_capex_df = pd.concat([factset_capex_int_v3_df, factset_capex_usc_v3_df])

In [ ]:
factset_capex_df.rename(columns={'ff_capex':            'Capital Expenditures (Total)',                  
'ff_capex_5ygr': 'Capital Spending - 5Yr Growth Rate',
'ff_capex_assets': 'Capital Expenditure percent Total Assets',
'ff_capex_fix': 'Capital Expenditures (Fixed Assets)',
'ff_capex_fix_assets': 'Capital Expediture percent Gross Fixed Assets',
'ff_capex_oth': 'Capital Expenditures (Other Assets)',
'ff_capex_ps_cf':      'Capital Expenditures Per Share',
'ff_capex_sales': 'Capital Expenditure percent Total Sales'       },
inplace=True)

In [ ]:
factset_capex_df.drop_duplicates(inplace=True)

In [ ]:
factset_capex_df['year'] = pd.DatetimeIndex(factset_capex_df['date']).year

In [ ]:
factset_capex_annual_availability_df = factset_capex_df[['isin', 'year', 'date']].groupby(by='isin').nunique()
factset_capex_annual_availability_df.hist(column='year', density=True)
plt.title('Distribution of the No. of Years per ISIN \n(Factset CAPEX Data)')
plt.xlabel('No. of Years')
plt.ylabel('Proportion of ISINs in Factset')
plt.xticks(np.arange(start=0, stop=12, step=1))
plt.show()

In [ ]:
factset_capex_annual_availability_df.hist(column='date', density=True, bins = 45)
plt.title('Distribution of the No. of Quarters per ISIN \n(Factset CAPEX Data)')
plt.xlabel('No. of Quarters')
plt.ylabel('Proportion of ISINs in Factset')
plt.xticks(np.arange(start=0, stop=47, step=5))
plt.show()

In [ ]:
factset_get_missing_value_df(factset_capex_df)

In [ ]:
factset_capex_df.to_excel("/content/drive/MyDrive/Factset_capex_data_with_currency_0716.xlsx")

### *Annual* EPS-related variables (Updated 8/11)

| Variable in FactSet Database          | Description                  |
|---------------------|-----------------------------------------------|
| ff_eps             | Earnings Per Share - Fiscal Period                     |
| ff_eps_dil_aft_xord    | Earnings Per Share (Diluted) After Extraordinary Items           |
| ff_eps_basic       | EPS - Basic - Before Extraordinaries                   |
| ff_eps_basic_gr    | EPS - Basic - Before Extras - percent Change           |
| ff_eps_contin_oper | Earnings Per Share from Continuing Operations - Fiscal |

In [ ]:
factset_eps_int_v3_df = db.raw_sql("SELECT isin, date, proper_name, currency, ff_eps, "
"ff_eps_dil_aft_xord, ff_eps_basic, ff_eps_basic_gr, ff_eps_contin_oper "
"FROM wrds_fund_af_int_v3 "
"WHERE isin in %(isins)s AND date >= %(start)s AND date <= %(end)s",
params=vars)

In [ ]:
factset_eps_usc_v3_df = db.raw_sql("SELECT isin, date, proper_name, currency, ff_eps, "
"ff_eps_dil_aft_xord, ff_eps_basic, ff_eps_basic_gr, ff_eps_contin_oper "
"FROM wrds_fund_af_usc_v3 "
"WHERE isin in %(isins)s AND date >= %(start)s AND date <= %(end)s",
params=vars)

In [ ]:
factset_eps_df = pd.concat([factset_eps_int_v3_df, factset_eps_usc_v3_df])

In [ ]:
factset_eps_df.drop_duplicates(inplace=True)

In [ ]:
factset_eps_df.rename(columns={'ff_eps':'Earnings Per Share - Fiscal Period',
'ff_eps_dil_aft_xord': 'Earnings Per Share (Diluted) After Extraordinary Items',           
'ff_eps_basic':        'EPS - Basic - Before Extraordinaries',                   
'ff_eps_basic_gr':     'EPS - Basic - Before Extras - percent Change',           
'ff_eps_contin_oper':  'Earnings Per Share from Continuing Operations - Fiscal'},
inplace=True)

In [ ]:
factset_eps_df['year'] = pd.DatetimeIndex(factset_eps_df['date']).year

In [ ]:
factset_get_missing_value_df(factset_eps_df)

In [ ]:
factset_eps_df.to_excel("/content/drive/MyDrive/factset_eps_data_with_currency_0716.xlsx")

## Annual Long Term Incentive Plan (ESOP) & Pensions-related variables

| Variable in FactSet Database |                   Description                   |
|:----------------------------:|:-----------------------------------------------:|
| ff_pfd_stk_esop              | Preferred Stock Issues for ESOP           |
| ff_pfd_stk_esop_gtd          | ESOP Guarantees - Preferred Stock               |
| ff_com_stk_esop              | ESOP Debt Guarantee                             |
| ff_pens_liabs_net            | Pension - Net Liability/(Asset)                 |
| ff_pens_bnfit_oblig_proj     | Pension - Projected Benefit Obligation          |
| ff_pens_bnfit_retir_post     | Pension/Post Retirement Benefits                |
| ff_pens_assets_oth           | Pension Plan - Asset Allocation - percent Other |
| ff_pens_assets_lt            | Pension - Long Term Assets                      |            |
| ff_pens_assets_emplr_contr   | Assets Employer Contributions                   |
| ff_pens_abo                  | Pension -Accumulated Benefit Obligation         |

In [ ]:
factset_ltip_int_v3_df = db.raw_sql("SELECT isin, date, proper_name, currency, ff_pfd_stk_esop, "
"ff_pfd_stk_esop_gtd, ff_com_stk_esop, ff_pens_liabs_net, ff_pens_bnfit_oblig_proj, ff_pens_bnfit_retir_post, "
"ff_pens_assets_oth, ff_pens_assets_lt, ff_pens_assets_emplr_contr, ff_pens_abo "
"FROM wrds_fund_af_int_v3 "
"WHERE isin in %(isins)s AND date >= %(start)s AND date <= %(end)s",
params=vars)

In [ ]:
factset_ltip_usc_v3_df = db.raw_sql("SELECT isin, date, proper_name, currency, ff_pfd_stk_esop, "
"ff_pfd_stk_esop_gtd, ff_com_stk_esop, ff_pens_liabs_net, ff_pens_bnfit_oblig_proj, ff_pens_bnfit_retir_post, "
"ff_pens_assets_oth, ff_pens_assets_lt, ff_pens_assets_emplr_contr, ff_pens_abo "
"FROM wrds_fund_af_usc_v3 "
"WHERE isin in %(isins)s AND date >= %(start)s AND date <= %(end)s",
params=vars)

In [ ]:
factset_ltip_df = pd.concat([factset_ltip_int_v3_df, factset_ltip_usc_v3_df])

In [ ]:
factset_ltip_df.rename(columns={'ff_pfd_stk_esop': 'Preferred Stock Issues for ESOP',                 
'ff_pfd_stk_esop_gtd':         'ESOP Guarantees - Preferred Stock',               
'ff_com_stk_esop':             'ESOP Debt Guarantee',                             
'ff_pens_liabs_net':           'Pension - Net Liability/(Asset)',                 
'ff_pens_bnfit_oblig_proj':    'Pension - Projected Benefit Obligation',          
'ff_pens_bnfit_retir_post':    'Pension/Post Retirement Benefits',                
'ff_pens_assets_oth':          'Pension Plan - Asset Allocation - percent Other', 
'ff_pens_assets_lt':           'Pension - Long Term Assets',                                  
'ff_pens_assets_emplr_contr':  'Assets Employer Contributions',                   
'ff_pens_abo':                 'Pension -Accumulated Benefit Obligation'         },
inplace=True)

In [ ]:
get_missing_value_df(factset_ltip_df)

In [ ]:
factset_ltip_df.to_excel("/content/drive/MyDrive/factset_ltip_data_with_currency_0716.xlsx")

### *Annual* Short Term & Long Term Variables/Roadmaps (Updated 8/11)

**Short-Term**: 

   
| Variable in FactSet Database |                   Description                   |
|:----------------------------:|:-----------------------------------------------:|
| ff_invest_st_tot | Total Short-Term Investments |        
| ff_cash_st | Cash & Short Term Investments |          
| ff_debt_serv | Debt Service Ratio - Incl St Debt | - Pretax             
| ff_debt_st | Short-Term Debt (incl. Current Portion | of LTD)       
| ff_liabs_curr | Total Current Liabilities |            
| ff_std_debt | Short Term Debt percent Total Debt |                  



In [ ]:
factset_st_int_v3_df = db.raw_sql("SELECT isin, date, proper_name, currency, "
"ff_invest_st_tot, ff_cash_st, ff_debt_serv, ff_debt_st, ff_std_debt, ff_liabs_curr "
"FROM wrds_fund_af_int_v3 "
"WHERE isin IN %(isins)s AND date >= %(start)s AND date <= %(end)s",
params=vars)

In [ ]:
factset_st_usc_v3_df = db.raw_sql("SELECT isin, date, proper_name, currency, "
"ff_invest_st_tot, ff_cash_st, ff_debt_serv, ff_debt_st, ff_std_debt, ff_liabs_curr "
"FROM wrds_fund_af_usc_v3 "
"WHERE isin IN %(isins)s AND date >= %(start)s AND date <= %(end)s",
params=vars)

In [ ]:
factset_st_df = pd.concat([factset_st_int_v3_df, factset_st_usc_v3_df])

In [ ]:
factset_st_df.rename(columns={'ff_invest_st_tot': "Total Short-Term Investments",
                              'ff_cash_st': 'Cash & Short Term Investments',
                              'ff_debt_serv': 'Debt Service Ratio - Incl Short Term Debt - Pretax',
                              'ff_debt_st': 'Short-Term Debt (incl. Current Portion of LTD)',
                              'ff_liabs_curr': 'Total Current Liabilities',
                              'ff_std_debt': 'Short Term Debt (% Total Debt)'}, inplace=True)

In [ ]:
factset_st_df.drop_duplicates(inplace=True)
factset_st_df['year'] = pd.DatetimeIndex(factset_st_df['date']).year

In [ ]:
factset_st_df.to_excel("/content/drive/MyDrive/factset_st_data_with_currency_0716.xlsx")

In [ ]:
factset_get_missing_value_df(factset_st_df)

**Long-Term**:                   
`ff_invest_lt`  Long term Investments                    
`ff_debt_lt` Long Term Debt         
`ff_debt_lt_iss_cf`	Issuance of LT Debt     
- Represents the amount received by the company from the issuance of long-term
debt (convertible and nonconvertible), increase in capitalized lease obligations, and
debt acquired from acquisitions.                   

`ff_debt_lt_reduct_cf` Reduction of Long Term Debt               
`ff_ins_lt_rsrv` Long Term Insurance Reserves    
`ff_liabs`	Long Term Liabilities           
`ff_liabs_ps`	Long Term Liabilities Per Share   (**Deprecated: Only annual data available**)                                  
`ff_ltd_tcap`	Long Term Debt percent Total Capital                                                    

In [ ]:
factset_lt_int_v3_df = db.raw_sql("SELECT isin, date, proper_name, currency, "
"ff_invest_lt, ff_debt_lt, ff_debt_lt_iss_cf, ff_debt_lt_reduct_cf, ff_ins_lt_rsrv, ff_liabs, ff_ltd_tcap "
"FROM wrds_fund_af_int_v3 "
"WHERE isin IN %(isins)s AND date >= %(start)s AND date <= %(end)s",
params=vars)

In [ ]:
factset_lt_usc_v3_df = db.raw_sql("SELECT isin, date, proper_name, currency, "
"ff_invest_lt, ff_debt_lt, ff_debt_lt_iss_cf, ff_debt_lt_reduct_cf, ff_ins_lt_rsrv, ff_liabs, ff_ltd_tcap "
"FROM wrds_fund_af_usc_v3 "
"WHERE isin IN %(isins)s AND date >= %(start)s AND date <= %(end)s",
params=vars)

In [ ]:
factset_lt_df = pd.concat([factset_lt_int_v3_df, factset_lt_usc_v3_df])

In [ ]:
factset_lt_df.rename(columns={'ff_invest_lt': 'Long term Investments',
                              'ff_debt_lt_tot': 'Total Long Term Debt',
                              'ff_debt_lt_iss_cf': 'Issuance of LT Debt',
                              'ff_debt_lt_reduct_cf': 'Reduction of Long Term Debt',
                              'ff_ins_lt_rsrv': 'Long Term Insurance Reserves',
                              'ff_liabs_ps': 'Long Term Liabilities Per Share',
                              'ff_ltd_tcap': 'Long Term Debt percent Total Capital'},
                     inplace=True)

In [ ]:
factset_lt_df.drop_duplicates(inplace=True)
factset_lt_df['year'] = pd.DatetimeIndex(factset_lt_df['date']).year

In [ ]:
factset_lt_df.to_excel("/content/drive/MyDrive/factset_lt_data_with_currency_0716.xlsx")

In [ ]:
factset_get_missing_value_df(factset_lt_df)

### Annual Healthcare-related Variables

| Variable in FactSet Database |                   Description                   |
|:----------------------------:|:-----------------------------------------------:|
| ff_hlth_abo |	Healthcare - Accumulated Benefit Obligation |                           
| ff_hlth_emplr_contr |	APBO- Health Employer Contributions |           
| ff_hlth_npp_exp |	Health Care Expense |                     
| ff_hlth_pbo |	Healthcare - Projected Benefit Obligation |                             
| ff_hlth_serv_cost |	Healthcare - Service Costs |

In [ ]:
factset_healthcare_int_v3_df = db.raw_sql("SELECT isin, date, proper_name, currency, "
"ff_hlth_abo, ff_hlth_emplr_contr, ff_hlth_npp_exp, ff_hlth_pbo, ff_hlth_serv_cost "
"FROM wrds_fund_af_int_v3 "
"WHERE isin IN %(isins)s AND date >= %(start)s AND date <= %(end)s",
params=vars)

In [ ]:
factset_healthcare_usc_v3_df = db.raw_sql("SELECT isin, date, proper_name, currency, "
"ff_hlth_abo, ff_hlth_emplr_contr, ff_hlth_npp_exp, ff_hlth_pbo, ff_hlth_serv_cost "
"FROM wrds_fund_af_usc_v3 "
"WHERE isin IN %(isins)s AND date >= %(start)s AND date <= %(end)s",
params=vars)

In [ ]:
factset_healthcare_df = pd.concat([factset_healthcare_int_v3_df, factset_healthcare_usc_v3_df])

In [ ]:
factset_healthcare_df.rename(columns={'ff_hlth_abo': 'Healthcare - Accumulated Benefit Obligation',
                                      'ff_hlth_emplr_contr': 'APBO - Health Employer Contributions',
                                       'ff_hlth_npp_exp': 'Health Care Expense',
                                       'ff_hlth_pbo': 'Healthcare - Projected Benefit Obligation',
                                       'ff_hlth_serv_cost': 'Healthcare - Service Costs'},
                             inplace=True)

In [ ]:
factset_healthcare_df['year'] = pd.DatetimeIndex(factset_healthcare_df['date']).year

In [ ]:
def factset_get_missing_value_df(df):
  # Print the no. of ISINs for which there is data
  num_isins = len(df['isin'].unique())
  isin_percent = (num_isins / 4147) * 100
  print('Of all the 4147 ISINs, there is data present for ' + str(num_isins) + ' ISINs (' + str(round(isin_percent, 2)) + '%).') 

  # Missingness by Year for Each Variable
  print('\n')
  print('Missingness by Year for Each Variable')
  missingness_by_year_df = (df.drop(columns=['isin', 'date', 'year', 'proper_name']).isnull().groupby(df['year'].astype(int)).sum() / (4147*4))*100
  display(missingness_by_year_df)

  # Missingness in an "ideal world" (If there was data for all 4147 ISINs by quarter for all 11 years) 
  print('\n')
  print('Missingness in an "ideal world" (If there was data for all 4147 ISINs by quarter for all 11 years)')
  percent_missing_ideal = (df.drop(columns=['isin', 'date', 'year', 'proper_name']).isna().sum() / (4147*4*11))*100
  missing_value_df = pd.DataFrame({'percent_missing_ideal': percent_missing_ideal})
  missing_value_df.rename(columns={'percent_missing_ideal': '% Missing Data (Out of All ISINs for All Years)'}, inplace=True)
  missing_value_df.sort_values(by='% Missing Data (Out of All ISINs for All Years)', inplace=True)
  display(missing_value_df)

factset_get_missing_value_df(factset_healthcare_df)

In [ ]:
factset_healthcare_df.to_excel("/content/drive/MyDrive/factset_healthcare_data_with_currency_0716.xlsx")

## BoardEx Data (Annual)



### BoardEx North America

***na_wrds_org_summary***    

This table contains a row for each director of an organization (identified by its ISIN).     


`genderratio` (Proportion of male directors)           
`nationalitymix` (Proportion of Directors from different countries)              
`noquals` (Avg number of qualifications at undergraduate level & above for directors)        
`network size` (Network size of selected indivdual on the board of directors: no. of overlaps through employment, other activities, & education)                     
`successsion` (Succession Factor: Number of Directors that have left a role as a proportion of average number of Directors for the preceding reporting period at the Annual Report Date selected
Succession Factor)              
`attrition` (Attrition Rate: Number of Directors that have left a role as a proportion of average number of Directors for the preceding reporting period at the Annual Report Date)  
`timeretirement` (Time to Retirement of the selected individual, assuming a retirement age of 70)           
`timeinco` (Time in Company)  
`stdevage` (Standard Deviation of Age)           
`stdevtimebrd` (Standard Deviation of Time on Board)                         
`stdevtimeinco` (Standard Deviation of Time in Company)               


In [ ]:
isin_list = pd.read_csv("/content/drive/MyDrive/isin_list.txt", header = None)

In [ ]:
import datetime
start_date = datetime.date(2010, 1, 1)
end_date = datetime.date(2020, 12, 31)

In [ ]:
vars = {'orgs': tuple(isin_list[0].unique()), 'start': start_date, 'end': end_date}

In [ ]:
# Getting board data for North America ISINs 

boardex_na_df = db.raw_sql("SELECT annualreportdate AS date, isin, boardname, hocountryname, currency, genderratio, nationalitymix, noquals, succession, attrition, "
           "networksize, numberdirectors, timeinco, timeretirement, stdevage, stdevtimebrd, stdevtimeinco "
           "FROM na_wrds_org_summary "
           "WHERE isin IN %(orgs)s AND annualreportdate >= %(start)s AND annualreportdate <= %(end)s ",
           params = vars)

In [ ]:
boardex_na_df.head(n=10)

In [ ]:
boardex_na_df['date'].sort_values()

In [ ]:
# Note that different companies published their annual reports at different dates (some in Dec, some in Nov)

boardex_na_df['date'].value_counts(dropna=False)

In [ ]:
boardex_na_df.describe()

In [ ]:
boardex_na_missing_df = get_missing_value_df(boardex_na_df)
boardex_na_missing_df

### BoardEx Europe

In [ ]:
boardex_eur_df = db.raw_sql("SELECT annualreportdate AS date, isin, boardname, hocountryname, currency, genderratio, nationalitymix, noquals, succession, attrition, "
           "networksize, numberdirectors, timeinco, timeretirement, stdevage, stdevtimebrd, stdevtimeinco "
           "FROM eur_wrds_org_summary "
           "WHERE isin IN %(orgs)s AND annualreportdate >= %(start)s AND annualreportdate <= %(end)s ",
           params = vars)

In [ ]:
boardex_eur_df.head()

In [ ]:
boardex_eur_missing_df = get_missing_value_df(boardex_eur_df)
boardex_eur_missing_df

In [ ]:
boardex_eur_df.describe()

### BoardEx UK

In [ ]:
boardex_uk_df = db.raw_sql("SELECT annualreportdate AS date, isin, boardname, hocountryname, currency, genderratio, nationalitymix, noquals, succession, attrition, "
           "networksize, numberdirectors, timeinco, timeretirement, stdevage, stdevtimebrd, stdevtimeinco "
           "FROM uk_wrds_org_summary "
           "WHERE isin IN %(orgs)s AND annualreportdate >= %(start)s AND annualreportdate <= %(end)s ",
           params = vars)

In [ ]:
boardex_uk_df.describe()

In [ ]:
boardex_uk_missing_df = get_missing_value_df(boardex_uk_df)
boardex_uk_missing_df

In [ ]:
boardex_uk_df['date'].value_counts()

### BoardEx Rest of World

In [ ]:
boardex_row_df = db.raw_sql("SELECT annualreportdate AS date, isin, boardname, hocountryname, currency, genderratio, nationalitymix, noquals, succession, attrition, "
           "networksize, numberdirectors, timeinco, timeretirement, stdevage, stdevtimebrd, stdevtimeinco "
           "FROM row_wrds_org_summary "
           "WHERE isin IN %(orgs)s AND annualreportdate >= %(start)s AND annualreportdate <= %(end)s ",
           params = vars)

In [ ]:
# Examining the Geographical Distributions of the "Rest of World" ISINs
boardex_row_df['hocountryname'].value_counts()

In [ ]:
boardex_row_df.describe()

In [ ]:
boardex_row_missing_df = get_missing_value_df(boardex_row_df)
boardex_row_missing_df

### Remarks about BoardEx

Remarks about BoardEx:      
- Since different organisations in different regions defined the start/end dates of their financial year differently, for simplicity, we extract the year from the date during which the annual report was published.

- There are 4147 unique ISINs in the FCLT dataset provided. 

BoardEx North America has data for 1034 of those ISINs (around 25% of our targeted ISINs).            
BoardEx Europe has data for 592 ISINs (around 14%).           
BoardEX UK has data for 141 ISINs (3.4%).      
BoardEx Rest Of World has data for 1699 ISINs (around 41%).          
The total no. of ISINs represented across the different regions is 3467, but 47 ISINs appeared in more than one region, so altogether, BoardEx has data for 3420 *distinct* ISINs (82.5% of the ISINs we are interested in). 

- Across all the regions, attrition rate is the variable with the most missing data.


In [ ]:
# Concatenating the four dataframes across the different regions 

boardex_df = pd.concat([boardex_na_df, boardex_eur_df, boardex_uk_df, boardex_row_df])

In [ ]:
# Find common ISINs across the different regions

from functools import reduce
import numpy as np

na_isins = boardex_na_df['isin'].unique()
eur_isins = boardex_eur_df['isin'].unique()
uk_isins = boardex_uk_df['isin'].unique()
row_isins = boardex_row_df['isin'].unique()

reduce(np.intersect1d, (na_isins, eur_isins, uk_isins, row_isins))

In [ ]:
# Find all the ISINs that appear in two regions
# https://stackoverflow.com/questions/52408910/python-pairwise-intersection-of-multiple-lists-then-sum-up-all-duplicates

from itertools import combinations_with_replacement

# Dictionary indexed by region where the values are the ISIN lists
boardex_isins = {'North America': na_isins, 'Europe': eur_isins, 'UK': uk_isins, 'Rest of World': row_isins}

In [ ]:
# Use combinations_with_replacement to compute all pairwise intersections of the ISINs lists,
# and put the results into a dictionary indexed by a frozenset of the region combinations,
# where the values are the cardinality of the intersection between the two lists

isin_set = { frozenset([a[0], b[0]]) : len(a[1] & b[1]) for a, b in combinations_with_replacement([(k, set(v)) for k, v in boardex_isins.items()], 2) if a != b }

In [ ]:
# Print the ISINs that appeared in a pairwise intersection of regions

region_pairs = list(isin_set.keys())
for i in np.arange(len(region_pairs)):
  print("ISINs appearing in both ", end="")
  print(tuple(dict.fromkeys(region_pairs[i])), end = ': ') 
  print(list(isin_set.values())[i])

In [ ]:
# There are 47 ISINs that appeared in exactly two regions. 
sum(isin_set.values())

In [ ]:
# However, there are only 46 ISINs that we overcounted -- this means that one ISIN appeared in three or more regions!
sum([len(na_isins), len(eur_isins), len(uk_isins), len(row_isins)]) - len(boardex_df['isin'].unique())

In [ ]:
boardex_df.to_excel("/content/drive/MyDrive/Boardex_diversity_data_with_currency_0716.xlsx")

### LTIP data from BoardEx

Organization Summary:      
**`valLTIPHeld`: Value of Long Term Incentive Plans Held (in 000s)**         
Value of LTIPs held at the end of the report for the individual based on the closing stock price of the Annual Report Date selected

**`LTIPValue`: Value of LTIP Awarded in last year (in 000s)**        
Value of LTIPs awarded during the period based on the closing stock price of the Annual Report Date selected   


**`PenEmpCon`: Employers Defined Contribution (in 000s)** 
Employers Defined Retirement/Pension Contribution for each Director in a report period


**`TotRemPeriod`: Total Awards for the Remuneration Period (in 000s)**
Total awards for the period

**`PerfTotal`: LTIP Value / Total Awards for the Period**                   
(Performance to total - Ratio of Value of LTIPs Held to Total Compensation for the period)  

**`TotalDirectComp`: Total Direct Compensation (000s) (Salary+Bonus+Other+PenEmpCont)**       
(Total Direct Compensation equals Salary plus Bonus plus Other Compensation plus Employers Devined Retirement/Pension Contribution for the period)


In [ ]:
boardex_ltip_na_df = db.raw_sql("SELECT annualreportdate AS date, isin, boardname, hocountryname, currency, "
          "valLTIPHeld, LTIPValue, PenEmpCon, TotRemPeriod, PerfTotal, TotalDirectComp "
          "FROM na_wrds_org_summary "
          "WHERE isin IN %(orgs)s AND annualreportdate >= %(start)s AND annualreportdate <= %(end)s", 
           params=vars)

In [ ]:
boardex_ltip_eur_df = db.raw_sql("SELECT annualreportdate AS date, isin, boardname, hocountryname, currency, "
          "valLTIPHeld, LTIPValue, PenEmpCon, TotRemPeriod, PerfTotal, TotalDirectComp "
          "FROM eur_wrds_org_summary "
          "WHERE isin IN %(orgs)s AND annualreportdate >= %(start)s AND annualreportdate <= %(end)s", 
           params=vars)

In [ ]:
boardex_ltip_uk_df = db.raw_sql("SELECT annualreportdate AS date, isin, boardname, hocountryname, currency, "
          "valLTIPHeld, LTIPValue, PenEmpCon, TotRemPeriod, PerfTotal, TotalDirectComp "
          "FROM uk_wrds_org_summary "
          "WHERE isin IN %(orgs)s AND annualreportdate >= %(start)s AND annualreportdate <= %(end)s", 
           params=vars)

In [ ]:
boardex_ltip_row_df = db.raw_sql("SELECT annualreportdate AS date, isin, boardname, hocountryname, currency, "
          "valLTIPHeld, LTIPValue, PenEmpCon, TotRemPeriod, PerfTotal, TotalDirectComp "
          "FROM row_wrds_org_summary "
          "WHERE isin IN %(orgs)s AND annualreportdate >= %(start)s AND annualreportdate <= %(end)s", 
           params=vars)

In [ ]:
boardex_ltip_df = pd.concat([boardex_ltip_na_df, boardex_ltip_eur_df, boardex_ltip_uk_df, boardex_ltip_row_df])

In [ ]:
boardex_ltip_df.rename(columns={'valltipheld': "Value of LTIPs Held (in 000s)",
                                'ltipvalue': 'Value of LTIP Awarded in Last Year (in 000s)',
                                'penempcon': 'Employers Defined Contribution (in 000s)',
                                'totremperiod': 'Total Awards for the Remuneration Period (in 000s)',
                                'perftotal': 'LTIP Value / Total Awards for the Period',
                                'totaldirectcomp': 'Total Direct Compensation (in 000s)'},
                       inplace=True)

In [ ]:
boardex_ltip_df.to_excel("/content/drive/MyDrive/Boardex_ltip_data_with_currency_0716.xlsx")

In [ ]:
boardex_ltip_df.describe()

In [ ]:
boardex_ltip_missing_df = get_missing_value_df(boardex_ltip_df)
boardex_ltip_missing_df

## ISS RiskMetrics Data (Annual)

ISS uses the CUSIP (and not the ISIN) to identify securities, so we need to first convert our list of ISINs to CUSIPs.

In [ ]:
cusip_candidates = tuple(isin[2:11] for isin in isin_list[0].unique())

In [ ]:
start_date = datetime.date(2010, 1, 1)
end_date = datetime.date(2020, 12, 31)

iss_vars = {'cusips': cusip_candidates, 'start': start_date, 'end': end_date}

**`iss_va_shareholder`** (Voting results for shareholder proposals)         
https://wrds-www.wharton.upenn.edu/data-dictionary/iss_va_shareholder/


In [ ]:
iss_shareholder_proposals_df = db.raw_sql("SELECT meeting_date, data_year, cusip, resolution, sponsor_type, requirement, passed, levelofsupport, "
            "support_for_against, support_for_against_abstain "
            "FROM va_proposals WHERE cusip IN %(cusips)s AND data_year >= 2010 AND data_year <= 2020", 
           params=iss_vars)

In [ ]:
iss_shareholder_proposals_df.groupby(by=['cusip', 'data_year']).agg('count')

In [ ]:
iss_shareholder_proposals_df['cusip'].unique().shape

In [ ]:
get_missing_value_df_annual(iss_shareholder_proposals_df)

**`vavoteresults`** (North America Voting Results)               
https://wrds-www.wharton.upenn.edu/data-dictionary/iss_va_vote_us/vavoteresults/

**`globalvoteresults`** (Global Voting Results)        
https://wrds-www.wharton.upenn.edu/pages/get-data/institutional-shareholder-services-iss/voting-analytics/company-vote-results-global/


In [ ]:
iss_vote_na_df = db.raw_sql("SELECT meetingdate, cusip, name, countryofinc, itemdesc, mgmtrec, sponsor, outstandingshare, "
                            "votedabstain, votedagainst, votedfor, votedwithheld, voterequirement, voteresult "
                            "FROM vavoteresults "
                            "WHERE cusip IN %(cusips)s AND meetingdate >= %(start)s AND meetingdate <= %(end)s",
                            params=iss_vars)

In [ ]:
iss_vote_na_df['cusip'].unique().shape

In [ ]:
iss_vote_global_df = db.raw_sql("SELECT meetingdate, cusip, name, countryofinc, itemdesc, mgmtrec, sponsor, outstandingshare, "
                            "votedabstain, votedagainst, votedfor, votedwithheld, voterequirement, voteresult "
                            "FROM globalvoteresults "
                            "WHERE cusip IN %(cusips)s AND meetingdate >= %(start)s AND meetingdate <= %(end)s",
                            params=iss_vars)

In [ ]:
iss_vote_global_df['cusip'].unique().shape

In [ ]:
iss_vote_global_df.describe()

In [ ]:
import numpy as np

np.intersect1d(iss_vote_na_df['cusip'].unique(), iss_vote_global_df['cusip'].unique()).shape

In [ ]:
# Concatenate NA and Global Vote Results dataframes

iss_vote_df = pd.concat([iss_vote_na_df, iss_vote_global_df])

In [ ]:
iss_vote_df.head()

In [ ]:
iss_vote_df.shape

In [ ]:
iss_vote_df['cusip'].unique().shape

In [ ]:
get_missing_value_df_annual(iss_vote_df)

**`rmgovernance`** Table (Governance data on Takeover Defenses):
https://wrds-www.wharton.upenn.edu/data-dictionary/risk_governance/rmgovernance/


**`year`** Year         
**`coname`** 	Company Name         
**`cusip`** CUSIP          
**`maj_vote_required`** Majority Vote Required             
**`maj_vote_requirement`** Majority Vote Requirement             

**`charter_amend_votepcnt`** Vote % Required to Amend Charter             
**`confvote`** Presence of Confidential Voting              
**`cumvote`** Cumulative Voting allowed               
**`labylw`** Whether Shareholders have the Ability to Amend ByLaws (boolean)         
**`lachtr`** Whether Shareholders have the Ability to Amend Charter (boolean)             
**`law_amend_votepcnt`** Vote % Required to Amend ByLaws              
**`ppill`** Whether Company has a Poison Pill Provision              
**`lspmt`** Whether Shareholders have the Ability to Call Special Meeting (boolean)               
**`lwcnst`** Whether Shareholders have the Ability to Act by Written Consent            
**`spl_meet_votepcnt`** Vote % Required to Call Special Meeting                    
**`written_consent_votepcnt`** Vote % Required for Written Consent            
**`supermajor_pcnt`** Vote Required to Approve Merger (%)             
**`uneqvote`** Presence of Unequal Voting Rights (boolean)              
**`resign_require`** Resignation Requirement                
**`oo_stakeholder`** Opt out of Stakeholder Law                

In [ ]:
iss_governance_df = db.raw_sql("SELECT year, coname, cusip, maj_vote_required, maj_vote_requirement, charter_amend_votepcnt, confvote, cumvote, labylw, lachtr, law_amend_votepcnt, "
                              "lspmt, lwcnst, ppill, spl_meet_votepcnt, written_consent_votepcnt, supermajor_pcnt, uneqvote, resign_require, oo_stakeholder "
                              "FROM rmgovernance "
                              "WHERE cusip IN %(cusips)s AND year >= 2010 AND year <= 2020",
                              params=iss_vars)

In [ ]:
iss_governance_df['cusip'].unique().shape

In [ ]:
iss_governance_df.head()

In [ ]:
get_missing_value_df_annual(iss_governance_df)

In [ ]:
iss_governance_df['cusip'].unique().shape

# Exporting Raw Dataframes

In [ ]:
with pd.ExcelWriter("/content/mnt/MyDrive/FCLT_Annual_ESG_Variables_0811.xlsx") as writer:
  ws_financial_df.to_excel(writer, "Financial Data (WorldScope)")
  factset_payable_df.to_excel(writer, "Financial Data (FactSet)")
  factset_capex_df.to_excel(writer, "CAPEX Data (FactSet)")
  factset_eps_df.to_excel(writer, "EPS Data (FactSet)")
  factset_st_df.to_excel(writer, "Short-Term Metrics (FactSet)")
  factset_lt_df.to_excel(writer, "Long-Term Variables (FactSet)")
  


# with pd.ExcelWriter("/content/mnt/MyDrive/FCLT_ESG_Variables_0706.xlsx") as writer:
#   refinitiv_env_df.to_excel(writer, "Environmental Data (Refinitiv)")
#   refinitiv_emp_df.to_excel(writer, "Employee Metrics (Refinitiv)")
#   factset_payable_df.to_excel(writer, "Financial Data (FactSet)")
#   factset_capex_df.to_excel(writer, "CAPEX Data (FactSet)")
#   factset_eps_df.to_excel(writer, "EPS Data (FactSet)")
#   factset_ltip_df.to_excel(writer, "LTIP & Pensions Data (FactSet)")
#   factset_st_df.to_excel(writer, "Short-Term Metrics (FactSet)")
#   factset_lt_df.to_excel(writer, "Long-Term Variables (FactSet)")
#   factset_healthcare_df.to_excel(writer, "Healthcare Data (FactSet)")
#   boardex_df.to_excel(writer, "Board Diversity Data (BoardEx)")
#   boardex_ltip_df.to_excel(writer, "LTIP Data (BoardEx)")
#   iss_shareholder_proposals_df.to_excel(writer, "Shareholder Proposals (ISS)")
#   iss_vote_df.to_excel(writer, "Voting Data (ISS)")
#   iss_governance_df.to_excel(writer, "Corporate Governance Data (ISS)")

# Interpolation of Annual Data into Quarterly Data (To be completed)

In [ ]:
filepath = "/content/drive/MyDrive/FCLT_ESG_Variables_0706.xlsx"

In [ ]:
# Rapidly find the sheet names of the Excel file containing the annual data we collected earlier
# Code adapted from https://stackoverflow.com/questions/17977540/pandas-looking-up-the-list-of-sheets-in-an-excel-file

from zipfile import ZipFile
from bs4 import BeautifulSoup  # you also need to install "lxml" for the XML parser

with ZipFile(filepath) as zipped_file:
    summary = zipped_file.open(r'xl/workbook.xml').read()
soup = BeautifulSoup(summary, "xml")
sheets = [sheet.get("name") for sheet in soup.find_all("sheet")]

In [ ]:
sheets

In [ ]:
factset_ltip_index = sheets.index('LTIP & Pensions Data (FactSet)')

In [ ]:
# Interpolate Factset Annual LTIP Data so that we have quarterly observations
factset_ltip_annual_df = pd.read_excel(filepath, sheet_name = 5)

In [ ]:
pd.date_range(start='1/1/2010', end='12/31/2020', freq='Q')

In [ ]:
factset_ltip_annual_df.dtypes

In [ ]:
factset_ltip_annual_df.drop(columns='Unnamed: 0', inplace=True)
factset_ltip_annual_df.drop_duplicates(inplace=True)

In [ ]:
factset_ltip_annual_df.dropna().head(n=5)

In [ ]:
quarters = pd.date_range(start='1/1/2010', end='12/31/2020', freq='Q')
quarters

In [ ]:
factset_ltip_annual_df.set_index('date').index

In [ ]:
test_df = factset_ltip_annual_df.reindex(factset_ltip_annual_df.set_index('date').index.union(quarters)).groupby('isin', group_keys=False) \
  .apply(lambda df: df.resample('Q').interpolate()).reset_index()

In [ ]:
# Date_df contains all quarters from factset_ltip_annual_df (including missing ones)
date_df = factset_ltip_annual_df.reindex(factset_ltip_annual_df.set_index('date').index.union(quarters)).reset_index().drop(columns=['date']).rename(columns={'index': 'date'})

# TODO: figure out how to merge date-df with  factset_ltip_quarterly_df while interpolating missing data

In [ ]:
# OLD CODE BELOW

# TODO: figure out how to upsample from yearly data to quarterly data 

# factset_ltip_annual_df['date'] = pd.to_datetime(factset_ltip_annual_df['date'])


# Adapted code from https://stackoverflow.com/questions/38641596/extend-data-frame-with-missing-dates-by-group-and-interpolate

factset_ltip_quarterly_df = factset_ltip_annual_df.set_index('date').groupby('isin', group_keys=False) \
  .apply(lambda df: df.resample('Q').interpolate()).reset_index()

In [ ]:
factset_ltip_quarterly_df.head(n=50)

In [ ]:
factset_ltip_quarterly_df[['isin', 'proper_name']] = factset_ltip_quarterly_df[['isin', 'proper_name']].fillna(method="ffill")

In [ ]:
factset_ltip_earliest_df = factset_ltip_quarterly_df.groupby('isin').agg('min').sort_values(by='date').reset_index()

In [ ]:
factset_ltip_earliest_df[factset_ltip_earliest_df['date'] != datetime.datetime(2010, 3, 31)]

In [ ]:
datetime.datetime(2010, 3, 31)

In [ ]:
factset_ltip_quarterly_df.shape

In [ ]:
factset_ltip_quarterly_df['isin'].unique().shape

In [ ]:
factset_ltip_quarterly_df['date'].value_counts()

Useful links:             
Reading xlsx files quicker (converting xlsx to csv in parallel):      
https://stackoverflow.com/questions/9884353/xls-to-csv-converter       
https://towardsdatascience.com/read-excel-files-with-python-1000x-faster-407d07ad0ed8              


Upsampling data using Pandas:      
https://stackoverflow.com/questions/45306105/pandas-convert-yearly-to-monthly       

https://stackoverflow.com/questions/57703538/typeerror-only-valid-with-datetimeindex-timedeltaindex-or-periodindex-but-got                   
(latter link is for resolving type errors while upsampling)                 